In [1]:
import os
import shutil
import numpy as np
import statistics as stat
import pandas

# Chemin du répertoire contenant les fichiers
repertoire = '../BDD/'
fichier_forme = "../Forme/Global/graphiques_global.xlsx"
annee_courante = 2024
annee_diplome = 2023

# Parcourir tous les fichiers dans le répertoire
for fichier in os.listdir(repertoire):
    # Vérifier si le fichier commence par "CGE-SciencesPoLyon"
    if fichier.startswith(f"CGE-SciencesPoLyon_PGE{annee_courante}"):
        nouveau_nom = os.path.join(repertoire, "data_n.xlsx")
        ancien_nom = os.path.join(repertoire, fichier)
        shutil.move(ancien_nom, nouveau_nom)
        print(f'Le fichier {fichier} a été renommé en data_n.xlsx')
    if fichier.startswith(f"CGE-SciencesPoLyon_PGE{annee_courante - 1}"):
        nouveau_nom = os.path.join(repertoire, "data_n-1.xlsx")
        ancien_nom = os.path.join(repertoire, fichier)
        shutil.move(ancien_nom, nouveau_nom)
        print(f'Le fichier {fichier} a été renommé en data_n-1.xlsx')
    if fichier.startswith(f"ExportDiplomesPourInsertionPro({annee_diplome - 1}-{annee_diplome})"):
        nouveau_nom = os.path.join(repertoire, "bdd_n.xls")
        ancien_nom = os.path.join(repertoire, fichier)
        shutil.move(ancien_nom, nouveau_nom)
        print(f'Le fichier {fichier} a été renommé en bdd_n.xls')
    if fichier.startswith(f"ExportDiplomesPourInsertionPro({annee_diplome - 2}-{annee_diplome - 1})"):
        nouveau_nom = os.path.join(repertoire, "bdd_n-1.xls")
        ancien_nom = os.path.join(repertoire, fichier)
        shutil.move(ancien_nom, nouveau_nom)
        print(f'Le fichier {fichier} a été renommé en bdd_n-1.xls')
    if fichier.startswith(f"ExportDiplomesPourInsertionPro({annee_diplome - 3}-{annee_diplome - 2})"):
        nouveau_nom = os.path.join(repertoire, "bdd_n-2.xls")
        ancien_nom = os.path.join(repertoire, fichier)
        shutil.move(ancien_nom, nouveau_nom)
        print(f'Le fichier {fichier} a été renommé en bdd_n-2.xls')

for fichier in os.listdir("../Forme/Global/"):
    if fichier.endswith(".xlsx"):
        chemin_fichier = os.path.join("../Forme/Global/", fichier)
        os.remove(chemin_fichier)
        print(f"Le fichier {fichier} a été supprimé.")


Le fichier graphiques_global.xlsx a été supprimé.


In [2]:
# Import des données de l'enquête
D = pandas.read_excel(repertoire + "data_n.xlsx",index_col=0)
#D = D[D['215. PROGRESSION'] == 'Terminé'] # Choix de seulement ceux qui ont bien terminé l'enquête

# Import des données de la base de données de Sciences Po Lyon (cyclades) besoin du package xlrd
BD = pandas.read_excel(repertoire + "bdd_n.xls",index_col=0)
BD_n_moins_1 = pandas.read_excel(repertoire + "bdd_n-1.xls",index_col=0)
BD_n_moins_2 = pandas.read_excel(repertoire + "bdd_n-2.xls",index_col=0)

# Import des données de l'enquête précédante
DO = pandas.read_excel(repertoire + "data_n-1.xlsx",index_col=0)


In [3]:
def get_keys_from_value(d, value):
    """
    in : 
        d : dictionnaire; où chercher la clé correspondante
        value : int, string; valeur liée à la clé recherchée dans le dictionnaire
    return :
        liste
    
    Fonction qui renvoie la clés ou les clés du dictionnaire correspondant à la valeur passée en paramètre.
    """
    return [k for k, v in d.items() if v == value]

def calcul_pourcentage(nombre, total, nb_virgule=0, en_pourcent=True):
    """
    in :
        nombre : entier; part du total
        total : entier;
        nb_virgule : entier; nombre de virgules voulu après le resultat
        en_pourcent : booléen; défini si le résultat est renvoyé en % ou en flottant
    return :
        entier ou float

    Fonction qui renvoie le pourcentage d'un nombre dans son total (le résultat peut être donné en % ou en flottant)
    """
    pourcentage = 0

    if nb_virgule < 0:
        if en_pourcent:
            pourcentage = (nombre / total) * 100
        else:
            pourcentage = nombre / total
    else:
        if en_pourcent:
            pourcentage = round((nombre / total) * 100, nb_virgule)
        else:
            pourcentage = round(nombre / total, nb_virgule)

    return pourcentage

def addition_dict(dict1, dict2):
    """
    in : 
        dict1, dict2 : dictionnaires; avec la même structure de clés
    return :
        Dictionnaire

        Fonction qui renvoie un dictionnaire contenant la somme sur chaque valeur de clé des 2 dictionnaires.
    """
    sum_dict = {}
    cles = []

    for d in [dict1, dict2]:
        for c in d.keys():
            if c not in cles:
                cles.append(c)

    for c in cles:
        sum_c = 0
        for d in [dict1, dict2]:
            if c in d.keys():
                sum_c += d[c]
        sum_dict.update({c : sum_c})

    return sum_dict

def nombre_apparition_DF(colonnes, cles, DFs):
    """
    in : 
        colonnes : liste de string; noms de colonnes où calculer le nombre
        cles : liste de string; cles dont on veut le pourcentage
        DFs : liste de DataFrame; dataframe où trouver les colonnes
        nb_v : Entier; Nombre de chiffres après la virgule sur le résultat (-1 = aucun arrondis)
    return : 
        Entier

    Fonction qui renvoie le nombre d'apparition d'une ou plusieurs valeurs dans une ou plusieurs colonnes 
    d'un dataframe pandas. (Ignore les valeurs NaN présente dans les colonnes du dataframe DF)
    """
    sum_dict = {}
    total = 0

    for cl in cles:
        sum_dict.update({cl : 0})
    
    for DF in DFs:
        for c in colonnes:
            try:
                sum_dict = addition_dict(sum_dict, DF[c].value_counts().to_dict())
            except:
                continue

    for cl in cles:
        total += sum_dict[cl]
        
    return total

def moyenne_apparition_DF(colonnes, DFs, nb_v=0):
    """
    in : 
        colonnes : liste de string; noms de colonnes où calculer la moyenne
        DFs : liste de DataFrame; dataframe où trouver les colonnes
        nb_v : Entier; Nombre de chiffres après la virgule sur le résultat (-1 = aucun arrondis)
    return : 
        Entier

    Fonction qui renvoie la moyenne d'une ou plusieurs colonnes d'un dataframe pandas. 
    (Ignore les valeurs NaN présente dans les colonnes du dataframe DF)
    """
    sum_dict = {}
    tab_val = []
    total = 0
    
    for DF in DFs:
        for c in colonnes:
            if c in DF.columns:
                sum_dict = addition_dict(sum_dict, DF[c].value_counts().to_dict())
    
    for k, v in sum_dict.items():
        for i in range(v):
            tab_val.append(k)
    
    if nb_v == -1:
        moyenne = stat.mean(tab_val)
    else:
        moyenne = round(stat.mean(tab_val), nb_v)
        
    return moyenne

def mediane_apparition_DF(colonnes, DFs, nb_v=0):
    """
    in : 
        colonnes : liste de string; noms de colonnes où calculer la médiane
        DFs : liste de DataFrame; dataframe où trouver les colonnes
        nb_v : Entier; Nombre de chiffres après la virgule sur le résultat (-1 = aucun arrondis)
    return : 
        Entier

    Fonction qui renvoie la médiane d'une ou plusieurs colonnes d'un dataframe pandas. 
    (Ignore les valeurs NaN présente dans les colonnes du dataframe DF)
    """
    sum_dict = {}
    tab_val = []
    total = 0
    
    for DF in DFs:
        for c in colonnes:
            if c in DF.columns:
                sum_dict = addition_dict(sum_dict, DF[c].value_counts().to_dict())
    
    for k, v in sum_dict.items():
        for i in range(v):
            tab_val.append(k)
    
    if nb_v == -1:
        mediane = stat.median(tab_val)
    else:
        mediane = round(stat.median(tab_val), nb_v)
        
    return mediane

def pourcentage_apparition_DF(colonnes, cles, DFs, nb_virgule=0, en_pourcent=True):
    """
    in : 
        colonnes : liste de string; noms de colonnes où calculer le pourcentage
        cles : liste de string; cles dont on veut le pourcentage
        DFs : liste de DataFrame; dataframe où trouver les colonnes
        nb_v : Entier; Nombre de chiffres après la virgule sur le résultat (-1 = aucun arrondis)
        en_pourcent : Booléen; Détermine si le résultat est de la forme % ou à virgule
    return : 
        Entier

    Fonction qui renvoie le pourcentage d'apparition d'une ou plusieurs valeurs dans une ou plusieurs colonnes 
    d'un dataframe pandas. (Ignore les valeurs NaN présente dans les colonnes du dataframe DF)
    """
    sum_dict = {}
    total = 0
    pourcentage = 0
    
    for DF in DFs:
        for c in colonnes:
            try:
                sum_dict = addition_dict(sum_dict, DF[c].value_counts().to_dict())
            except:
                continue

    total = sum(sum_dict.values())
    
    if en_pourcent == True:
        for cl in cles:
            try:
                pourcentage += calcul_pourcentage(sum_dict[cl], total, -1)
            except:
                continue
    else:
        for cl in cles:
            try:
                pourcentage += calcul_pourcentage(sum_dict[cl], total, -1, False)
            except:
                continue
    pourcentage = round(pourcentage, nb_virgule)

    return pourcentage

def valeurs_uniques(colonnes, DF):
    """
    in :
        colonnes : liste; contenant le nom des colonnes où trouver toutes les valeurs uniques
        DF : DataFrame; où trouver les colonnes
    return : 
        liste
    
    Fonction qui renvoie la liste des valeurs uniques dans une ou plusieurs colonnes.
    """
    valeurs_uniques = {}
    
    for c in colonnes:
        valeurs_uniques = addition_dict(valeurs_uniques, DF[c].value_counts().to_dict())
    
    return list(valeurs_uniques.keys())


def creation_graphique_HF(colonnes, DF, DF_F, DF_H):
    """
    in :
        colonnes : liste; des colonnes où faires tous les pourcentage pour le graphique
        DF : DataFrame; où trouver les colonnes
    return : 
        dictionnaire
        
    Fonction qui renvoie un dictionnaire pour créer un graphique de type femme homme.
    """
    vals = valeurs_uniques(colonnes, DF)
    
    d_F = {}
    d_H = {}
    d = {
        "Sexe" : ["Femmes", "Hommes"]
    }

    for v in vals:
        for c in colonnes:
            d_F = addition_dict(d_F, {v : pourcentage_apparition_DF([c], [v], [DF_F], 3, False)})
            d_H = addition_dict(d_H, {v : pourcentage_apparition_DF([c], [v], [DF_H], 3, False)})
        d.update({v : [d_F[v], d_H[v]]})
        
    return d



def filtre_DF(colonnes, filtres, DF):
    """
    in :
        colonnes : liste de String; contenant les colonnes à filtrer respectivement par les filtres
        filtres : liste de String; contenant les filtres à appliquer respectivement sur les colonnes
        DF : DataFrame; à filtrer
    return :
        DataFrame
        
    Fonction qui applique 1 ou plusieurs filtres sur un DataFrame et le retourne
    """
    if len(colonnes) != len(filtres):
        raise Exception("Les 2 listes en paramètres ne sont pas de la même taille")
        
    DF_tmp = DF.copy()
    
    for i in range(len(colonnes)):
        try:
            DF_tmp = DF_tmp[DF_tmp[colonnes[i]] == filtres[i]]
        except:
            print("Problème de clé")
            
    return DF_tmp

def sauvegarde_excel(sorties, fichier, feuille):
    """
    in : 
        sorties : liste de DataFrame; chaque dataframe sera enregistrer sur une feuille
        fichier : String; nom de fichier excel dans lequel enregistrer les feuilles
        feuille : String; nom des feuilles qui iront dans le fichier excel
    
    Procédure qui enregistre des DataFrames dans un fichier excel. 
    Chaque DataFrame est enregistré sur la feuille donnée en paramètre avec un numéro de 0 à n.
    """
    
    for i in range(len(sorties)):
        sortie = sorties[i]
        
        if os.path.exists(fichier):
            with pandas.ExcelWriter(fichier, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
                sortie.to_excel(writer, sheet_name=feuille, index=False)
        else:
            with pandas.ExcelWriter(fichier, engine='openpyxl') as writer:
                sortie.to_excel(writer, sheet_name=feuille, index=False)

In [4]:
# Calcul chiffres forts

duree_CDD = [2, 3, 4, 5, 6, 8, 9, 10, 12, 16, 18, 24, 32, 36, 72]
tab_duree = [0] * len(duree_CDD)

for index, row in D.iterrows():
    if isinstance(row["32. EmploiContratFranceDuree"], float) and np.isnan(row["32. EmploiContratFranceDuree"]):
        continue
    tab_duree[duree_CDD.index(row["32. EmploiContratFranceDuree"])] += 1
                  
total_duree = sum(tab_duree)

print(tab_duree)
print(total_duree)

# Pourcentage Durée CDD

pourcentage_duree = [0] * len(duree_CDD)

for i in range(len(duree_CDD)):
    pourcentage_duree[i] = round(tab_duree[i] / total_duree, 3)

print(pourcentage_duree)

pourcentage_plus_6mois = sum(pourcentage_duree[4:])

print(pourcentage_plus_6mois)

[1, 2, 3, 4, 7, 2, 2, 3, 21, 1, 3, 1, 1, 7, 1]
59
[0.017, 0.034, 0.051, 0.068, 0.119, 0.034, 0.034, 0.051, 0.356, 0.017, 0.051, 0.017, 0.017, 0.119, 0.017]
0.832


In [5]:
# Graphique chiffres forts

donnees = dict(zip(duree_CDD, pourcentage_duree))

sortie = pandas.DataFrame([donnees])

sortie.insert(0, "durée en mois", "pourcentage")

sortie["6 mois et plus"] = round(pourcentage_plus_6mois * 100, 0)

sortie.head()

,durée en mois,2,3,4,5,6,8,9,10,12,16,18,24,32,36,72,6 mois et plus
0,pourcentage,0.017,0.034,0.051,0.068,0.119,0.034,0.034,0.051,0.356,0.017,0.051,0.017,0.017,0.119,0.017,83.0


In [6]:
# Modification fichier excel
 
sheet_name = "graph_0-1"
sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [7]:
# Calcul taux de répondants
nombre_repondant = D.shape[0]
nombre_diplome = BD.shape[0]

# Pourcentage de répondants
pourcentage_repondant = round(nombre_repondant / nombre_diplome, 3)
print(pourcentage_repondant)

0.833


In [8]:
# Graphique taux de répondants

annee = [2021, 2022, 2023]
nombre_de_diplomes = [273, 295, nombre_diplome]
taux_de_reponse = [213, 229, nombre_repondant]
pourcentage = ["78%", "77,6%", str(round(pourcentage_repondant * 100,1)).replace('.', ',') + "%"]

donnees = {
    'Année': annee,
    'Nombre de diplômés': nombre_de_diplomes,
    'Taux de réponse': taux_de_reponse,
    'Pourcentage': pourcentage
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,Année,Nombre de diplômés,Taux de réponse,Pourcentage
0,2021,273,213,78%
1,2022,295,229,"77,6%"
2,2023,269,224,"83,3%"


In [9]:
# Modification fichier excel

sheet_name = "graph_1-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [10]:
# Calcul pourcentage homme femme PPR répondants

nombre_homme_total = 80
nombre_femme_total = 189
nombre_PPR_total = 0 #PPR = préfère pas répondre
nombre_homme_repondant = 0
nombre_femme_repondant = 0
nombre_PPR_repondant = 0

for index, row in D.iterrows():
    if row['4. IdentiteSexeVerifie'] == 'Homme':
        nombre_homme_repondant += 1
    elif row['4. IdentiteSexeVerifie'] == 'Femme':
        nombre_femme_repondant += 1
    else:
        nombre_PPR_repondant += 1

# Nombre répondants
print(f'h: {nombre_homme_repondant}; f: {nombre_femme_repondant}; ppr: {nombre_PPR_repondant}')

# Pourcentage répondants
pourcentage_femme_repondant = round(nombre_femme_repondant / nombre_repondant, 3)
pourcentage_homme_repondant = round(nombre_homme_repondant / nombre_repondant, 3)
pourcentage_PPR_repondant = round(nombre_PPR_repondant / nombre_repondant, 3)

print(f'h: {pourcentage_homme_repondant}; f: {pourcentage_femme_repondant}; ppr: {pourcentage_PPR_repondant}')

# Pourcentage total
pourcentage_femme_total = round(nombre_femme_total / nombre_diplome, 3)
pourcentage_homme_total = round(nombre_homme_total / nombre_diplome, 3)
pourcentage_PPR_total = round(nombre_PPR_total / nombre_diplome, 3)

print(f'h: {pourcentage_homme_total}; f: {pourcentage_femme_total}; ppr: {pourcentage_PPR_total}')


h: 67; f: 155; ppr: 2
h: 0.299; f: 0.692; ppr: 0.009
h: 0.297; f: 0.703; ppr: 0.0


In [11]:
# Graphique pourcentage hommme femme PPR répondants

etiquette = ["Parmi les répondants à l'enquête", "Parmi les diplômés"]
femmes = [pourcentage_femme_repondant, pourcentage_femme_total]
hommes = [pourcentage_homme_repondant, pourcentage_homme_total]
PPR = [pourcentage_PPR_repondant, pourcentage_PPR_total]

donnees = {
    'etiquette_hf': etiquette,
    'Femmes': femmes,
    'Hommes': hommes,
    'Ne souhaite pas répondre': PPR
}

sortie = pandas.DataFrame(donnees)
sortie.head()


,etiquette_hf,Femmes,Hommes,Ne souhaite pas répondre
0,Parmi les répondants à l'enquête,0.692,0.299,0.009
1,Parmi les diplômés,0.703,0.297,0.000


In [12]:
# Modification fichier excel

sheet_name = "graph_1-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [13]:
# Calucl répondants et diplômés selon le secteur de 4ème année

nombre_secteur_AI_repondant = 0 # Affaires Internationales
nombre_secteur_AP_repondant = 0 # Affaires Publiques
nombre_secteur_COM_repondant = 0 # Communication
nombre_secteur_TER_repondant = 0 # Territoires

nombre_secteur_AI_total = 84
nombre_secteur_AP_total = 32
nombre_secteur_COM_total = 46
nombre_secteur_TER_total = 13

nombre_secteur_total = nombre_secteur_AI_total + nombre_secteur_AP_total + nombre_secteur_COM_total + nombre_secteur_TER_total

for index, row in D.iterrows():
    if row['263. Ecole_4emeAnneeSecteur'] == "Affaires Internationales":
        nombre_secteur_AI_repondant += 1
    elif row['263. Ecole_4emeAnneeSecteur'] == "Affaires Publiques":
        nombre_secteur_AP_repondant += 1
    elif row['263. Ecole_4emeAnneeSecteur'] == "Communication":
        nombre_secteur_COM_repondant += 1
    elif row['263. Ecole_4emeAnneeSecteur'] == "Territoires":
        nombre_secteur_TER_repondant += 1
        
nombre_secteur_total_repondant = nombre_secteur_AI_repondant + nombre_secteur_AP_repondant + nombre_secteur_COM_repondant + nombre_secteur_TER_repondant

print(f'AI: {nombre_secteur_AI_repondant}; AP: {nombre_secteur_AP_repondant}; COM: {nombre_secteur_COM_repondant}; TER: {nombre_secteur_TER_repondant}')

# Pourcentage total secteurs 4ème
pourcentage_secteur_AI_total = round(nombre_secteur_AI_total / nombre_secteur_total, 3)
pourcentage_secteur_AP_total = round(nombre_secteur_AP_total / nombre_secteur_total, 3)
pourcentage_secteur_COM_total = round(nombre_secteur_COM_total / nombre_secteur_total, 3)
pourcentage_secteur_TER_total = round(nombre_secteur_TER_total / nombre_secteur_total, 3)

print(f'AI: {pourcentage_secteur_AI_total}; AP: {pourcentage_secteur_AP_total}; COM: {pourcentage_secteur_COM_total}; TER: {pourcentage_secteur_TER_total}')

# Pourcentage répondants secteurs 4ème
pourcentage_secteur_AI_repondant = round(nombre_secteur_AI_repondant / nombre_secteur_total_repondant, 3)
pourcentage_secteur_AP_repondant = round(nombre_secteur_AP_repondant / nombre_secteur_total_repondant, 3)
pourcentage_secteur_COM_repondant = round(nombre_secteur_COM_repondant / nombre_secteur_total_repondant, 3)
pourcentage_secteur_TER_repondant = round(nombre_secteur_TER_repondant / nombre_secteur_total_repondant, 3)

print(f'AI: {pourcentage_secteur_AI_repondant}; AP: {pourcentage_secteur_AP_repondant}; COM: {pourcentage_secteur_COM_repondant}; TER: {pourcentage_secteur_TER_repondant}')



AI: 85; AP: 55; COM: 44; TER: 19
AI: 0.48; AP: 0.183; COM: 0.263; TER: 0.074
AI: 0.419; AP: 0.271; COM: 0.217; TER: 0.094


In [14]:
# Graphique pourcentage diplômés répondants selon le secteur de 4ème année

donnees = {
    "etiquette_secteur_4eme" :  ["Parmi les diplômés", "Parmi les répondants à l'enquête",],
    "Affaires Internationales" : [pourcentage_secteur_AI_total, pourcentage_secteur_AI_repondant],
    "Affaires Publiques" : [pourcentage_secteur_AP_total, pourcentage_secteur_AP_repondant],
    "Communication" : [pourcentage_secteur_COM_total, pourcentage_secteur_COM_repondant],
    "Territoires" : [pourcentage_secteur_TER_total, pourcentage_secteur_TER_repondant]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,etiquette_secteur_4eme,Affaires Internationales,Affaires Publiques,Communication,Territoires
0,Parmi les diplômés,0.480,0.183,0.263,0.074
1,Parmi les répondants à l'enquête,0.419,0.271,0.217,0.094


In [15]:
# Modification fichier excel

  
sheet_name = "graph_1-3"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [16]:
# Graphique pourcentage secteur des diplômés en 4ème année

donnees = {
    "etiquette" : [2021, 2022, 2023],
    "Affaires Internationales" : [0.404, 0.44, pourcentage_secteur_AI_total],
    "Affaires Publiques" : [0.298, 0.324, pourcentage_secteur_AP_total],
    "Communication" : [0.232, 0.164, pourcentage_secteur_COM_total],
    "Territoires" : [0.066, 0.072, pourcentage_secteur_TER_total]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,etiquette,Affaires Internationales,Affaires Publiques,Communication,Territoires
0,2021,0.404,0.298,0.232,0.066
1,2022,0.440,0.324,0.164,0.072
2,2023,0.480,0.183,0.263,0.074


In [17]:
# Modification fichier excel

  
sheet_name = "graph_2-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [18]:
# Calucl diplômés selon le secteur de 4ème année (Hommes, Femmes)

# Femmes
nombre_secteur_AI_F = 82
nombre_secteur_AP_F = 32
nombre_secteur_COM_F = 46
nombre_secteur_TER_F = 13

nombre_femme_total = nombre_secteur_AI_F + nombre_secteur_AP_F + nombre_secteur_COM_F + nombre_secteur_TER_F

# Hommes
nombre_secteur_AI_H = 21
nombre_secteur_AP_H = 28
nombre_secteur_COM_H = 12
nombre_secteur_TER_H = 7

nombre_homme_total = nombre_secteur_AI_H + nombre_secteur_AP_H + nombre_secteur_COM_H + nombre_secteur_TER_H

# Pourcentage F selon secteurs 4ème
pourcentage_secteur_AI_F = round(nombre_secteur_AI_F / nombre_femme_total, 3)
pourcentage_secteur_AP_F = round(nombre_secteur_AP_F / nombre_femme_total, 3)
pourcentage_secteur_COM_F = round(nombre_secteur_COM_F / nombre_femme_total, 3)
pourcentage_secteur_TER_F = round(nombre_secteur_TER_F / nombre_femme_total, 3)

print(f'AI: {pourcentage_secteur_AI_F}; AP: {pourcentage_secteur_AP_F}; COM: {pourcentage_secteur_COM_F}; TER: {pourcentage_secteur_TER_F}')

# Pourcentage H selon secteurs 4ème
pourcentage_secteur_AI_H = round(nombre_secteur_AI_H / nombre_homme_total, 3)
pourcentage_secteur_AP_H = round(nombre_secteur_AP_H / nombre_homme_total, 3)
pourcentage_secteur_COM_H = round(nombre_secteur_COM_H / nombre_homme_total, 3)
pourcentage_secteur_TER_H = round(nombre_secteur_TER_H / nombre_homme_total, 3)

print(f'AI: {pourcentage_secteur_AI_H}; AP: {pourcentage_secteur_AP_H}; COM: {pourcentage_secteur_COM_H}; TER: {pourcentage_secteur_TER_H}')



AI: 0.474; AP: 0.185; COM: 0.266; TER: 0.075
AI: 0.309; AP: 0.412; COM: 0.176; TER: 0.103


In [19]:
# Graphique pourcentage secteur des diplômés en 4ème année (Hommes Femmes)

donnees = {
    "etiquette" : ["Femmes", "Hommes"],
    "Affaires Internationales" : [pourcentage_secteur_AI_F, pourcentage_secteur_AI_H],
    "Affaires Publiques" : [pourcentage_secteur_AP_F, pourcentage_secteur_AP_H],
    "Communication" : [pourcentage_secteur_COM_F, pourcentage_secteur_COM_H],
    "Territoires" : [pourcentage_secteur_TER_F, pourcentage_secteur_TER_H]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,etiquette,Affaires Internationales,Affaires Publiques,Communication,Territoires
0,Femmes,0.474,0.185,0.266,0.075
1,Hommes,0.309,0.412,0.176,0.103


In [20]:
# Modification fichier excel

  
sheet_name = "graph_2-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [21]:
BD['Spécialité 5A'].unique()


array(["Développement, Ingénierie de Projets et Coopération à l'International",
       'Globalisation et gouvernance', 'Carrières Publiques',
       'Spécialité',
       "Management des actions culturelles à l'international", nan,
       'Affaires Internationales Asiatiques',
       'Affaires européennes : entreprises et institutions',
       'Conduite de projets et développement durable des territoires',
       'Management des services publics et des partenariats public / privé',
       'Communication, culture et institutions', 'Journalisme',
       'Politiques et innovations sociales des territoires'], dtype=object)

In [22]:
# Calcul spécialités de 5ème année et master
nom_specialite = [x for x in BD['Spécialité 5A'].unique().tolist() if x == x] #Permet d'enlever la colonne vide

nombre_par_specialite = [[0] * len(nom_specialite) for _ in range(3)] 
print(nombre_par_specialite)
          
tab_bdd = [BD_n_moins_2, BD_n_moins_1, BD]  

nombre_total_specialite = [0] * 3

for i in range(3):
    for index, row in (tab_bdd[i].iterrows()):
        if isinstance(row["Spécialité 5A"], float) and np.isnan(row["Spécialité 5A"]):
            continue
        nombre_par_specialite[i][nom_specialite.index(row['Spécialité 5A'])] += 1
    nombre_total_specialite[i] = sum(nombre_par_specialite[i])    

print(nom_specialite)
print(nombre_par_specialite)
print(nombre_total_specialite)

# Pourcentage

pourcentage_par_specialite = [[0] * len(nom_specialite) for _ in range(3)]

print(pourcentage_par_specialite)

for i in range(3):
    for j in range(len(nom_specialite)):
        pourcentage_par_specialite[i][j] = round(nombre_par_specialite[i][j] / nombre_total_specialite[i], 3)

print(pourcentage_par_specialite)

# Tableau de correspondance des noms et des acronymes
correspondance = {
    "globalisation et gouvernance": "2G",
    "affaires européennes : entreprises et institutions": "A2EI",
    "affaires internationales asiatiques" :  "AFASIA",
    "journalisme" : "Journalisme",
    "analyses des politiques publiques": "APP",
    "carrières publiques": "CAPU",
    "communication, culture et institutions": "ComCI",
    "conduite de projets et développement durable des territoires": "CopTER",
    "evaluation et pilotage des politiques publiques": "EPPP",
    "développement, ingénierie de projets et coopération à l'international": "DIPCI",
    "journalisme, médias et territoires": "Journalisme",
    "management des actions culturelles à l'international": "MACI",
    "management des services publics et des partenariats public / privé": "MSP3P",
    "politiques et innovations sociales des territoires": "PIST",
    "politiques publiques de l'alimentation et gestion des risques sanitaires": "PAGERS",
    "administrateur des institutions de recherche et de diffusion des connaissances": "ADMIRE",
    "asie orientale contemporaine": "ASIOC",
    "histoire de la pensée politique": "HPP",
    "politiques européennes - voisinages de l'ue": "AlterEurope",
    "gouvernance et politiques urbaines alternatives": "AlterVilles",
    "spécialité": "Mast & spé ext"
}

# Transformation du tableau nom en accronymes
for i in range(len(nom_specialite)):
    if str.lower(nom_specialite[i]) in correspondance:
        nom_specialite[i] = correspondance[str.lower(nom_specialite[i])]
    else:
        print(f"spécialité :{nom_specialite[i]} inconnue")
    
print(nom_specialite)

    

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
["Développement, Ingénierie de Projets et Coopération à l'International", 'Globalisation et gouvernance', 'Carrières Publiques', 'Spécialité', "Management des actions culturelles à l'international", 'Affaires Internationales Asiatiques', 'Affaires européennes : entreprises et institutions', 'Conduite de projets et développement durable des territoires', 'Management des services publics et des partenariats public / privé', 'Communication, culture et institutions', 'Journalisme', 'Politiques et innovations sociales des territoires']
[[19, 23, 17, 114, 18, 0, 12, 10, 21, 21, 13, 4], [32, 22, 28, 112, 15, 0, 23, 11, 18, 15, 14, 3], [27, 22, 13, 103, 19, 10, 14, 13, 9, 16, 10, 5]]
[272, 293, 261]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0.07, 0.085, 0.062, 0.419, 0.066, 0.0, 0.044, 0.037, 0.077, 0.077, 

In [23]:
donnees = {
    "Spécialités" : nom_specialite,
    2021 : pourcentage_par_specialite[0],
    2022 : pourcentage_par_specialite[1],
    2023 : pourcentage_par_specialite[2]
}

sortie = pandas.DataFrame(donnees)

sortie.head(14)

,Spécialités,2021,2022,2023
0,DIPCI,0.070,0.109,0.103
1,2G,0.085,0.075,0.084
2,CAPU,0.062,0.096,0.050
3,Mast & spé ext,0.419,0.382,0.395
4,MACI,0.066,0.051,0.073
5,AFASIA,0.000,0.000,0.038
6,A2EI,0.044,0.078,0.054
7,CopTER,0.037,0.038,0.050
8,MSP3P,0.077,0.061,0.034
9,ComCI,0.077,0.051,0.061


In [24]:
# Modification fichier excel

  
sheet_name = "graph_3-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [25]:
# Calcul les entrées à Sciences Po Lyon
nombre_entre_4e = 0 # Sur concours 4ème année
nombre_entre_2e = 0 # Sur concours 2ème année
nombre_entre_1e = 0 # Sur concours 1ère année

for index, row in D.iterrows():
    if row['260. Ecole_EntreeIEP'] == 'Sur concours 4ème année':
        nombre_entre_4e += 1
    elif row['260. Ecole_EntreeIEP'] == 'Sur concours 2ème année':
        nombre_entre_2e += 1
    elif row['260. Ecole_EntreeIEP'] == 'Sur concours 1ère année':
        nombre_entre_1e += 1

nombre_total_entre = nombre_entre_4e + nombre_entre_2e + nombre_entre_1e

print(f"4e : {nombre_entre_4e}; 2e : {nombre_entre_2e}; 1e : {nombre_entre_1e}")
print(nombre_total_entre)

# Pourcentage par année d'entrée
pourcentage_entre_4e = round(nombre_entre_4e / nombre_total_entre, 3)
pourcentage_entre_2e = round(nombre_entre_2e / nombre_total_entre, 3)
pourcentage_entre_1e = round(nombre_entre_1e / nombre_total_entre, 3)

print(f"4e : {pourcentage_entre_4e}; 2e : {pourcentage_entre_2e}; 1e : {pourcentage_entre_1e}")


4e : 41; 2e : 30; 1e : 130
201
4e : 0.204; 2e : 0.149; 1e : 0.647


In [26]:
# Graphique entrée à sciences po lyon par année

donnees = {
    "Sur concours 1ère année" : [pourcentage_entre_1e],
    "Sur concours 2ème année" : [pourcentage_entre_2e],
    "Sur concours 4ème année" : [pourcentage_entre_4e],
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,Sur concours 1ère année,Sur concours 2ème année,Sur concours 4ème année
0,0.647,0.149,0.204


In [27]:
# Modification fichier excel

  
sheet_name = "graph_3-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [28]:
# Calcul niveau d'étude avant Sciences Po

nombre_bac = 0
nombre_bac12 = 0 # BAC +1/+2
nombre_bac3plus = 0 #BAC + 3 et plus
nombre_prepa = 0

for index, row in D.iterrows():
    if row["257. Ecole_AvantIEPNiveauEtudes"] == 'BAC':
        nombre_bac += 1
    elif row["257. Ecole_AvantIEPNiveauEtudes"] == 'BAC + 2' or row["257. Ecole_AvantIEPNiveauEtudes"] == 'BAC + 1':
        if row["258. Ecole_AvantIEPFormation"] == "Classe préparatoire aux grandes écoles (CPGE)":
            nombre_prepa += 1
        else:
            nombre_bac12 += 1
    elif row["257. Ecole_AvantIEPNiveauEtudes"] == 'BAC +3 et plus':
        if row["258. Ecole_AvantIEPFormation"] == "Classe préparatoire aux grandes écoles (CPGE)":
            nombre_prepa += 1
        else:
            nombre_bac3plus += 1
            
nombre_total_niv_etude = nombre_bac + nombre_bac12 + nombre_bac3plus + nombre_prepa

print(f"BAC : {nombre_bac} ; BAC+1/+2 : {nombre_bac12} ; BAC +3/plus : {nombre_bac3plus} ; prepa : {nombre_prepa}")
print(nombre_total_niv_etude)

# Pourcentage niveau d'études avant sicences po

pourcentage_bac = round(nombre_bac / nombre_total_niv_etude, 3)
pourcentage_bac12 = round(nombre_bac12 / nombre_total_niv_etude, 3)
pourcentage_bac3plus = round(nombre_bac3plus / nombre_total_niv_etude, 3)
pourcentage_prepa = round(nombre_prepa / nombre_total_niv_etude, 3)

print(f"BAC : {pourcentage_bac} ; BAC+1/+2 : {pourcentage_bac12} ; BAC +3/plus : {pourcentage_bac3plus} ; prepa : {pourcentage_prepa}")

BAC : 69 ; BAC+1/+2 : 19 ; BAC +3/plus : 34 ; prepa : 81
203
BAC : 0.34 ; BAC+1/+2 : 0.094 ; BAC +3/plus : 0.167 ; prepa : 0.399


In [29]:
# Graphique niveau d'étude avant Sciences Po

donnees = {
    "Année" : [2021, 2022, 2023],
    "BAC" : [0.302, 0.319, pourcentage_bac],
    "Année préparatoire" : [0.344, 0.363, pourcentage_prepa],
    "BAC +1/+2" : [0.137, 0.137, pourcentage_bac12],
    "BAC + 3 et plus" : [0.208, 0.181, pourcentage_bac3plus]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,Année,BAC,Année préparatoire,BAC +1/+2,BAC + 3 et plus
0,2021,0.302,0.344,0.137,0.208
1,2022,0.319,0.363,0.137,0.181
2,2023,0.340,0.399,0.094,0.167


In [30]:
# Modification fichier excel

  
sheet_name = "graph_3-3"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [31]:
# Calcul diplôme d'établissement d'aire culturelle

nombre_diplome_EAC = 0 # Diplôme d'établissement d'aire culturelle 
nombre_cursus_para = 0 
nombre_DEEE = 0 # Diplôme d'Établissement d'Etudes Européennes (DEEE)
nombre_DEMAC = 0 # Diplôme d'Établissement du Monde Arabe Contemporain (DEMAC)
nombre_DEMEOC = 0 # Diplôme d'Établissement du Monde Extrême-Oriental Contemporain (DEMEOC)
nombre_DEASC = 0 # Diplôme d'Établissement sur l'Afrique Subsaharienne Contemporaine (DEASC)
nombre_DEALC = 0 # Diplôme d'Établissement sur l'Amérique Latine et les Caraïbes (DEALC)
nombre_DELUSA = 0 # Diplôme d'Établissement sur les Etats-Unis (DELUSA)
nombre_DERUSCO = 0 # Diplôme d'Établissement sur la Russie Contemporaine (DERUSCO)

for index, row in D.iterrows():
    if row["270. Ecole_DEACOuiNon"] == "Oui":
        nombre_diplome_EAC += 1
    if row["272. Ecole_CursusParalele"] == "Oui":
        nombre_cursus_para += 1
    if row["271. Ecole_DEACDiplome"] == "Diplôme d'Établissement d'Etudes Européennes (DEEE)":
        nombre_DEEE += 1
    elif row["271. Ecole_DEACDiplome"] == "Diplôme d'Établissement du Monde Arabe Contemporain (DEMAC)":
        nombre_DEMAC += 1
    elif row["271. Ecole_DEACDiplome"] == "Diplôme d'Établissement du Monde Extrême-Oriental Contemporain (DEMEOC)":
        nombre_DEMEOC += 1
    elif row["271. Ecole_DEACDiplome"] == "Diplôme d'Établissement sur l'Afrique Subsaharienne Contemporaine (DEASC)":
        nombre_DEASC += 1
    elif row["271. Ecole_DEACDiplome"] == "Diplôme d'Établissement sur l'Amérique Latine et les Caraïbes (DEALC)":
        nombre_DEALC += 1
    elif row["271. Ecole_DEACDiplome"] == "Diplôme d'Établissement sur les Etats-Unis (DELUSA)":
        nombre_DELUSA += 1
    elif row["271. Ecole_DEACDiplome"] == "Diplôme d'Établissement sur la Russie Contemporaine (DERUSCO)":
        nombre_DERUSCO += 1

        
print(f"nbr EAC : {nombre_diplome_EAC}; nbr cursus para : {nombre_cursus_para}; DEEE : {nombre_DEEE}; DEMAC : {nombre_DEMAC}; DEMEOC : {nombre_DEMEOC}; DEASC : {nombre_DEASC}; DEALC : {nombre_DEALC}; DELUSA : {nombre_DELUSA}; DERUSCO : {nombre_DERUSCO}")

# Pourcentage diplôme d'établissement d'aire culturelle

pourcentage_diplome_EAC = int(round(nombre_diplome_EAC / nombre_repondant * 100, 0))
pourcentage_cursus_para = int(round(nombre_cursus_para / nombre_repondant * 100, 0))
pourcentage_DEEE = round(nombre_DEEE / nombre_diplome_EAC, 3)
pourcentage_DEMAC = round(nombre_DEMAC / nombre_diplome_EAC, 3)
pourcentage_DEMEOC = round(nombre_DEMEOC / nombre_diplome_EAC, 3)
pourcentage_DEASC = round(nombre_DEASC / nombre_diplome_EAC, 3)
pourcentage_DEALC = round(nombre_DEALC / nombre_diplome_EAC, 3)
pourcentage_DELUSA = round(nombre_DELUSA / nombre_diplome_EAC, 3)
pourcentage_DERUSCO = round(nombre_DERUSCO / nombre_diplome_EAC, 3)

print(f"% EAC : {pourcentage_diplome_EAC}; % cursus para : {pourcentage_cursus_para}; DEEE : {pourcentage_DEEE}; DEMAC : {pourcentage_DEMAC}; DEMEOC : {pourcentage_DEMEOC}; DEASC : {pourcentage_DEASC}; DEALC : {pourcentage_DEALC}; DELUSA : {pourcentage_DELUSA}; DERUSCO : {pourcentage_DERUSCO}")


nbr EAC : 110; nbr cursus para : 35; DEEE : 25; DEMAC : 20; DEMEOC : 22; DEASC : 9; DEALC : 11; DELUSA : 15; DERUSCO : 8
% EAC : 49; % cursus para : 16; DEEE : 0.227; DEMAC : 0.182; DEMEOC : 0.2; DEASC : 0.082; DEALC : 0.1; DELUSA : 0.136; DERUSCO : 0.073


In [32]:
# Graphique diplôme d'établissement d'aire culturelle

donnees = {
    "Année" : [2021, 2022, 2023],
    "EAC" : [54, 42, pourcentage_diplome_EAC],
    "cursus para" : [20, 18, pourcentage_cursus_para],
    "Diplôme d'Établissement d'Etudes Européennes (DEEE)" : [0.157, 0.242, pourcentage_DEEE],
    "Diplôme d'Établissement du Monde Arabe Contemporain (DEMAC)" : [0.209, 0.116, pourcentage_DEMAC],
    "Diplôme d'Établissement du Monde Extrême-Oriental Contemporain (DEMEOC)" : [0.183, 0.179, pourcentage_DEMEOC],
    "Diplôme d'Établissement sur l'Afrique Subsaharienne Contemporaine (DEASC)" : [0.061, 0.011, pourcentage_DEASC],
    "Diplôme d'Établissement sur l'Amérique Latine et les Caraïbes (DEALC)" : [0.157, 0.2, pourcentage_DEALC],
    "Diplôme d'Établissement sur les Etats-Unis (DELUSA)" : [0.235, 0.179, pourcentage_DELUSA],
    "Diplôme d'Établissement sur les Mondes Post Soviétiques (DEMOPS)" : [0, 0.074, pourcentage_DERUSCO]
}

sortie = pandas.DataFrame(donnees)

sortie.head()


,Année,EAC,cursus para,Diplôme d'Établissement d'Etudes Européennes (DEEE),Diplôme d'Établissement du Monde Arabe Contemporain (DEMAC),Diplôme d'Établissement du Monde Extrême-Oriental Contemporain (DEMEOC),Diplôme d'Établissement sur l'Afrique Subsaharienne Contemporaine (DEASC),Diplôme d'Établissement sur l'Amérique Latine et les Caraïbes (DEALC),Diplôme d'Établissement sur les Etats-Unis (DELUSA),Diplôme d'Établissement sur les Mondes Post Soviétiques (DEMOPS)
0,2021,54,20,0.157,0.209,0.183,0.061,0.157,0.235,0.000
1,2022,42,18,0.242,0.116,0.179,0.011,0.200,0.179,0.074
2,2023,49,16,0.227,0.182,0.200,0.082,0.100,0.136,0.073


In [33]:
# Modification fichier excel

  
sheet_name = "graph_4-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [34]:
# Calcul diplôme d'établissement d'aire culturelle par secteur de formation

nombre_AI_oui = 0
nombre_AP_oui = 0
nombre_COM_oui = 0
nombre_TER_oui = 0

nombre_AI_non = 0
nombre_AP_non = 0
nombre_COM_non = 0
nombre_TER_non = 0

for index, row in D.iterrows():
    if row["270. Ecole_DEACOuiNon"] == "Oui":
        if row["269. Option_ScolariteFiliereFormation"] == "Affaires internationales (AI)":
            nombre_AI_oui += 1
        elif row["269. Option_ScolariteFiliereFormation"] == "Affaires publiques (AP)":
            nombre_AP_oui += 1
        elif row["269. Option_ScolariteFiliereFormation"] == "Communication (COM)":
            nombre_COM_oui += 1
        elif row["269. Option_ScolariteFiliereFormation"] == "Territoires (TER)":
            nombre_TER_oui += 1
    else:
        if row["269. Option_ScolariteFiliereFormation"] == "Affaires internationales (AI)":
            nombre_AI_non += 1
        elif row["269. Option_ScolariteFiliereFormation"] == "Affaires publiques (AP)":
            nombre_AP_non += 1
        elif row["269. Option_ScolariteFiliereFormation"] == "Communication (COM)":
            nombre_COM_non += 1
        elif row["269. Option_ScolariteFiliereFormation"] == "Territoires (TER)":
            nombre_TER_non += 1
            
print(f"OUI : AI : {nombre_AI_oui}; AP : {nombre_AP_oui}; COM : {nombre_COM_oui}; TER : {nombre_TER_oui}; NON :  AI : {nombre_AI_non}; AP : {nombre_AP_non}; COM : {nombre_COM_non}; TER : {nombre_TER_non};")
     
# Pourcentages diplôme d'établissement d'aire culturelle par secteur de formation

total_AI = nombre_AI_oui + nombre_AI_non
total_AP = nombre_AP_oui + nombre_AP_non
total_COM = nombre_COM_oui + nombre_COM_non
total_TER = nombre_TER_oui + nombre_TER_non

pourcentage_AI_oui = round(nombre_AI_oui / total_AI, 3)
pourcentage_AP_oui = round(nombre_AP_oui / total_AP, 3)
pourcentage_COM_oui = round(nombre_COM_oui / total_COM, 3)
pourcentage_TER_oui = round(nombre_TER_oui / total_TER, 3)

pourcentage_AI_non = round(nombre_AI_non / total_AI, 3)
pourcentage_AP_non = round(nombre_AP_non / total_AP, 3)
pourcentage_COM_non = round(nombre_COM_non / total_COM, 3)
pourcentage_TER_non = round(nombre_TER_non / total_TER, 3)

print(f"%OUI : AI : {pourcentage_AI_oui}; AP : {pourcentage_AP_oui}; COM : {pourcentage_COM_oui}; TER : {pourcentage_TER_oui}; %NON :  AI : {pourcentage_AI_non}; AP : {pourcentage_AP_non}; COM : {pourcentage_COM_non}; TER : {pourcentage_TER_non};")

OUI : AI : 46; AP : 27; COM : 21; TER : 14; NON :  AI : 37; AP : 26; COM : 22; TER : 7;
%OUI : AI : 0.554; AP : 0.509; COM : 0.488; TER : 0.667; %NON :  AI : 0.446; AP : 0.491; COM : 0.512; TER : 0.333;


In [35]:
# Graphique pourcentage diplôme d'établissement d'aire culturelle par secteur de formation

donnees = {
    "etiquette" : ["Non", "Oui"],
    "Affaires Internationales" : [pourcentage_AI_non, pourcentage_AI_oui],
    "Affaires Publiques" : [pourcentage_AP_non, pourcentage_AP_oui],
    "Communication" : [pourcentage_COM_non, pourcentage_COM_oui],
    "Territoires" : [pourcentage_TER_non, pourcentage_TER_oui]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,etiquette,Affaires Internationales,Affaires Publiques,Communication,Territoires
0,Non,0.446,0.491,0.512,0.333
1,Oui,0.554,0.509,0.488,0.667


In [36]:
# Modification fichier excel

  
sheet_name = "graph_4-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [37]:
# Calcul situation

nombre_activite_pro = 0 # En activité professionnelle
nombre_etude = 0 # En études / En formation
nombre_recherche_emploi = 0 # En recherche d'emploi
nombre_these = 0 # En thèse / PhD
nombre_volontariat = 0 # En volontariat (VIE, VIA, Service civique, etc.)
nombre_autre = 0 # Autre situation

for index, row in D.iterrows():
    if row["15. Situation"] == "En activité professionnelle":
        nombre_activite_pro += 1
    elif row["15. Situation"] == "En études / En formation":
        nombre_etude += 1
    elif row["15. Situation"] == "En recherche d'emploi":
        nombre_recherche_emploi += 1
    elif row["15. Situation"] == "En thèse / PhD":
        nombre_these += 1
    elif row["15. Situation"] == "En volontariat (VIE, VIA, Service civique, ...)":
        nombre_volontariat += 1
    elif row["15. Situation"] == "Autre situation":
        nombre_autre += 1

total_activite = nombre_activite_pro + nombre_etude + nombre_recherche_emploi + nombre_these + nombre_volontariat + nombre_autre 
        
print(f"total : {total_activite}; activite_pro : {nombre_activite_pro}; etude : {nombre_etude}; recherche emploi : {nombre_recherche_emploi}; these : {nombre_these}; volontariat : {nombre_volontariat}; autre : {nombre_autre}")

# Pourcentage situation

pourcentage_activite_pro = round(nombre_activite_pro / total_activite, 3)
pourcentage_etude = round(nombre_etude / total_activite, 3)
pourcentage_recherche_emploi = round(nombre_recherche_emploi / total_activite, 3)
pourcentage_these = round(nombre_these / total_activite, 3)
pourcentage_volontariat = round(nombre_volontariat / total_activite, 3)
pourcentage_autre = round(nombre_autre / total_activite, 3)

print(f"activite_pro : {pourcentage_activite_pro}; etude : {pourcentage_etude}; recherche emploi : {pourcentage_recherche_emploi}; these : {pourcentage_these}; volontariat : {pourcentage_volontariat}; autre : {pourcentage_autre}")




total : 219; activite_pro : 122; etude : 42; recherche emploi : 25; these : 3; volontariat : 14; autre : 13
activite_pro : 0.557; etude : 0.192; recherche emploi : 0.114; these : 0.014; volontariat : 0.064; autre : 0.059


In [38]:
# Graphique pourcentage situation diplômés

donnees = {
    "En activité professionnelle" : [pourcentage_activite_pro, round(pourcentage_activite_pro * 100, 0), nombre_activite_pro],
    "En études / En formation" : [pourcentage_etude, round(pourcentage_etude * 100, 0), nombre_etude],
    "En recherche d'emploi" : [pourcentage_recherche_emploi, round(pourcentage_recherche_emploi * 100, 0), nombre_recherche_emploi],
    "En thèse / PhD" : [pourcentage_these, round(pourcentage_these * 100, 0), nombre_these],
    "En volontariat (VIE, VIA, Service civique, etc.)" : [pourcentage_volontariat, round(pourcentage_volontariat * 100, 0), nombre_volontariat],
    "Autres situations" : [pourcentage_autre, round(pourcentage_autre * 100, 0), nombre_autre]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,En activité professionnelle,En études / En formation,En recherche d'emploi,En thèse / PhD,"En volontariat (VIE, VIA, Service civique, etc.)",Autres situations
0,0.557,0.192,0.114,0.014,0.064,0.059
1,56.000,19.000,11.000,1.000,6.000,6.000
2,122.000,42.000,25.000,3.000,14.000,13.000


In [39]:
# Modification fichier excel

  
sheet_name = "graph_5-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [40]:
# Graphique pourcentage diplôme d'établissement d'aire culturelle par secteur de formation

donnees = {
    "Année" : [2021, 2022, 2023],
    "En activité" : [0.737, 0.594, pourcentage_activite_pro],
    "En recherche d'emploi" : [0.047, 0.079, pourcentage_recherche_emploi],
    "En service civique ou VIE" : [0.033 ,0.092, pourcentage_volontariat]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,Année,En activité,En recherche d'emploi,En service civique ou VIE
0,2021,0.737,0.047,0.033
1,2022,0.594,0.079,0.092
2,2023,0.557,0.114,0.064


In [41]:
# Modification fichier excel

  
sheet_name = "graph_5-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [42]:
# Calcul situation selon secteur

secteur = ["Affaires Internationales", "Affaires Publiques", "Communication", "Territoires"]
nombre_activite_pro = [0] * 4 # En activité professionnelle
nombre_etude = [0] * 4 # En études / En formation
nombre_recherche_emploi = [0] * 4 # En recherche d'emploi
nombre_these = [0] * 4 # En thèse / PhD
nombre_volontariat = [0] * 4 # En volontariat (VIE, VIA, Service civique, etc.)
nombre_autre = [0] * 4 # Autres situations

for index, row in D.iterrows():
    if isinstance(row["263. Ecole_4emeAnneeSecteur"], float): # Vérification si la case est vide (NaN)
        continue
    i = secteur.index(row["263. Ecole_4emeAnneeSecteur"])
    
    if row["15. Situation"] == "En activité professionnelle":
        nombre_activite_pro[i] += 1
    elif row["15. Situation"] == "En études / En formation":
        nombre_etude[i] += 1
    elif row["15. Situation"] == "En recherche d'emploi":
        nombre_recherche_emploi[i] += 1
    elif row["15. Situation"] == "En thèse / PhD":
        nombre_these[i] += 1
    elif row["15. Situation"] == "En volontariat (VIE, VIA, Service civique, ...)":
        nombre_volontariat[i] += 1
    elif row["15. Situation"] == "Autre situation":
        nombre_autre[i] += 1

total_activite = [0] * len(secteur)
        
for i in range(len(secteur)):
    total_activite[i] = nombre_activite_pro[i] + nombre_etude[i] + nombre_recherche_emploi[i] + nombre_these[i] + nombre_volontariat[i] + nombre_autre[i] 
    
    print(f"total {secteur[i]} : {total_activite[i]}; activite_pro : {nombre_activite_pro[i]}; etude : {nombre_etude[i]}; recherche emploi : {nombre_recherche_emploi[i]}; these : {nombre_these[i]}; volontariat : {nombre_volontariat[i]}; autre : {nombre_autre[i]}")

# Pourcentage situation

pourcentage_activite_pro = [0] * 4 # En activité professionnelle
pourcentage_etude = [0] * 4 # En études / En formation
pourcentage_recherche_emploi = [0] * 4 # En recherche d'emploi
pourcentage_these = [0] * 4 # En thèse / PhD
pourcentage_volontariat = [0] * 4 # En volontariat (VIE, VIA, Service civique, etc.)
pourcentage_autre = [0] * 4 # Autres situations

for i in range(len(secteur)):
    pourcentage_activite_pro[i] = round(nombre_activite_pro[i] / total_activite[i], 3)
    pourcentage_etude[i] = round(nombre_etude[i] / total_activite[i], 3)
    pourcentage_recherche_emploi[i] = round(nombre_recherche_emploi[i] / total_activite[i], 3)
    pourcentage_these[i] = round(nombre_these[i] / total_activite[i], 3)
    pourcentage_volontariat[i] = round(nombre_volontariat[i] / total_activite[i], 3)
    pourcentage_autre[i] = round(nombre_autre[i] / total_activite[i], 3)
    
    print(f"ligne {secteur[i]}; activite_pro : {pourcentage_activite_pro[i]}; etude : {pourcentage_etude[i]}; recherche emploi : {pourcentage_recherche_emploi[i]}; these : {pourcentage_these[i]}; volontariat : {pourcentage_volontariat[i]}; autre : {pourcentage_autre[i]}")



total Affaires Internationales : 85; activite_pro : 43; etude : 20; recherche emploi : 11; these : 0; volontariat : 7; autre : 4
total Affaires Publiques : 55; activite_pro : 27; etude : 13; recherche emploi : 7; these : 3; volontariat : 2; autre : 3
total Communication : 44; activite_pro : 28; etude : 7; recherche emploi : 6; these : 0; volontariat : 0; autre : 3
total Territoires : 19; activite_pro : 14; etude : 0; recherche emploi : 1; these : 0; volontariat : 3; autre : 1
ligne Affaires Internationales; activite_pro : 0.506; etude : 0.235; recherche emploi : 0.129; these : 0.0; volontariat : 0.082; autre : 0.047
ligne Affaires Publiques; activite_pro : 0.491; etude : 0.236; recherche emploi : 0.127; these : 0.055; volontariat : 0.036; autre : 0.055
ligne Communication; activite_pro : 0.636; etude : 0.159; recherche emploi : 0.136; these : 0.0; volontariat : 0.0; autre : 0.068
ligne Territoires; activite_pro : 0.737; etude : 0.0; recherche emploi : 0.053; these : 0.0; volontariat : 

In [43]:
# Graphique pourcentage diplôme d'établissement d'aire culturelle par secteur de formation

donnees = {
    "Secteur" : secteur,
    "En activité professionnelle" : pourcentage_activite_pro,
    "En études / En formation" : pourcentage_etude,
    "En recherche d'emploi" : pourcentage_recherche_emploi,
    "En thèse / PhD" : pourcentage_these,
    "En volontariat (VIE, VIA, Service civique, etc.)" : pourcentage_volontariat,
    "Autres situations" : pourcentage_autre
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,Secteur,En activité professionnelle,En études / En formation,En recherche d'emploi,En thèse / PhD,"En volontariat (VIE, VIA, Service civique, etc.)",Autres situations
0,Affaires Internationales,0.506,0.235,0.129,0.000,0.082,0.047
1,Affaires Publiques,0.491,0.236,0.127,0.055,0.036,0.055
2,Communication,0.636,0.159,0.136,0.000,0.000,0.068
3,Territoires,0.737,0.000,0.053,0.000,0.158,0.053


In [44]:
# Modification fichier excel

  
sheet_name = "graph_6-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [45]:
# Calcul pourcentage 1er emploi

nombre_oui = 0
nombre_non = 0

for index, row in D.iterrows():
    if row["147. 1erEmploiDecritPrecedemment"] == "Oui":
        nombre_oui += 1
    elif row["147. 1erEmploiDecritPrecedemment"] == "Non":
        nombre_non += 1

nombre_total = nombre_oui + nombre_non

pourcentage_1er_emploi = int(round(nombre_oui / nombre_total * 100, 0))

print(f"% 1er emploi : {pourcentage_1er_emploi}; nbr oui : {nombre_oui}; nbr non : {nombre_non}")
    


% 1er emploi : 82; nbr oui : 97; nbr non : 21


In [46]:
# Graphique pourcentage diplôme d'établissement d'aire culturelle par secteur de formation

donnees = {
    "1er emploi" : [pourcentage_1er_emploi],
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,1er emploi
0,82


In [47]:
# Modification fichier excel

  
sheet_name = "graph_6-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [48]:
# Calcul temps avant 1er job après diplôme

immediatement = 0
moins_1_mois = 0
de_1a2_mois = 0
de_2a3_mois = 0
de_3a4_mois = 0
de_4a5_mois = 0
de_5a6_mois = 0
plus_6_mois = 0

for index, row in D.iterrows():
    if row["149. 1erEmploiLapsPourTrouverApresDiplome"] == "Emploi trouvé avant l'obtention de votre diplôme":
        immediatement += 1
    elif row["149. 1erEmploiLapsPourTrouverApresDiplome"] == "Moins d'un mois":
        moins_1_mois += 1
    elif row["149. 1erEmploiLapsPourTrouverApresDiplome"] == "De 1 à moins de 2 mois":
        de_1a2_mois += 1
    elif row["149. 1erEmploiLapsPourTrouverApresDiplome"] == "De 2 à moins de 3 mois":
        de_2a3_mois += 1
    elif row["149. 1erEmploiLapsPourTrouverApresDiplome"] == "De 3 à moins de 4 mois":
        de_3a4_mois += 1
    elif row["149. 1erEmploiLapsPourTrouverApresDiplome"] == "De 4 à moins de 5 mois":
        de_4a5_mois += 1
    elif row["149. 1erEmploiLapsPourTrouverApresDiplome"] == "De 5 à moins de 6 mois":
        de_5a6_mois += 1
    elif row["149. 1erEmploiLapsPourTrouverApresDiplome"] == "6 mois ou plus":
        plus_6_mois += 1
        
total_temps = immediatement + moins_1_mois + de_1a2_mois + de_2a3_mois + de_3a4_mois + de_4a5_mois + de_5a6_mois + plus_6_mois

print(f"total : {total_temps}; immediat : {immediatement}; moins 1 mois : {moins_1_mois}; de_1a2_mois : {de_1a2_mois}; de_2a3_mois : {de_2a3_mois}; de_3a4_mois : {de_3a4_mois}; de_4a5_mois : {de_4a5_mois}; de_5a6_mois : {de_5a6_mois}; plus_6_mois : {plus_6_mois}")

# Pourcentage temps avant 1er job après diplôme

pourcentage_immediatement = round(immediatement / total_temps, 3)
pourcentage_moins_1_mois = round(moins_1_mois / total_temps, 3)
pourcentage_de_1a2_mois = round(de_1a2_mois / total_temps, 3)
pourcentage_de_2a3_mois = round(de_2a3_mois / total_temps, 3)
pourcentage_de_3a4_mois = round(de_3a4_mois / total_temps, 3)
pourcentage_de_4a5_mois = round(de_4a5_mois / total_temps, 3)
pourcentage_de_5a6_mois = round(de_5a6_mois / total_temps, 3)
pourcentage_plus_6_mois = round(plus_6_mois / total_temps, 3)

print(f"%  immediat : {pourcentage_immediatement}; moins 1 mois : {pourcentage_moins_1_mois}; de_1a2_mois : {pourcentage_de_1a2_mois}; de_2a3_mois : {pourcentage_de_2a3_mois}; de_3a4_mois : {pourcentage_de_3a4_mois}; de_4a5_mois : {pourcentage_de_4a5_mois}; de_5a6_mois : {pourcentage_de_5a6_mois}; plus_6_mois : {pourcentage_plus_6_mois}")

total : 138; immediat : 79; moins 1 mois : 12; de_1a2_mois : 15; de_2a3_mois : 9; de_3a4_mois : 9; de_4a5_mois : 3; de_5a6_mois : 5; plus_6_mois : 6
%  immediat : 0.572; moins 1 mois : 0.087; de_1a2_mois : 0.109; de_2a3_mois : 0.065; de_3a4_mois : 0.065; de_4a5_mois : 0.022; de_5a6_mois : 0.036; plus_6_mois : 0.043


In [49]:
# Graphique temps avant 1er job après diplôme

moins_2_mois = pourcentage_moins_1_mois + pourcentage_de_1a2_mois
de_1a4_mois = pourcentage_moins_1_mois + pourcentage_de_1a2_mois + pourcentage_de_2a3_mois + pourcentage_de_3a4_mois
de_2a4_mois = pourcentage_de_2a3_mois + pourcentage_de_3a4_mois
de_4a6_mois = pourcentage_de_4a5_mois + pourcentage_de_5a6_mois

donnees = {
    "Immédiatement" : [pourcentage_immediatement, round(pourcentage_immediatement * 100, 1)],
    "Moins de 1 mois" : [pourcentage_immediatement + pourcentage_moins_1_mois, round(pourcentage_moins_1_mois * 100, 1)],
    "De 1 à 4 mois" : [pourcentage_immediatement + de_1a4_mois, round(de_1a4_mois * 100, 1)],
    "De 4 à 6 mois" : [pourcentage_immediatement + de_1a4_mois + de_4a6_mois, round(de_4a6_mois * 100, 1)],
    "Plus de 6 mois" : [1, round(pourcentage_plus_6_mois * 100, 1)],
    "Moins d'un mois" : [pourcentage_immediatement + pourcentage_moins_1_mois, round((pourcentage_immediatement + pourcentage_moins_1_mois) * 100, 0)],
    "total nombre temps" : [total_temps, total_temps]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,Immédiatement,Moins de 1 mois,De 1 à 4 mois,De 4 à 6 mois,Plus de 6 mois,Moins d'un mois,total nombre temps
0,0.572,0.659,0.898,0.956,1.0,0.659,138
1,57.200,8.700,32.600,5.800,4.3,66.000,138


In [50]:
# Modification fichier excel

  
sheet_name = "graph_7-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [51]:
# Calcul temps avant 1er job après diplôme (mediane et moyenne)

tab = []
total = 0
tab_secteur = [[] for _ in range(4)]

for index, row in D.iterrows():
    if isinstance(row["150. 1erEmploiLapsPourTrouverApresDiplomeMois"], float) and np.isnan(row["150. 1erEmploiLapsPourTrouverApresDiplomeMois"]):
        continue
    tab += [row["150. 1erEmploiLapsPourTrouverApresDiplomeMois"]]
    total += 1
    if row["263. Ecole_4emeAnneeSecteur"] == "Affaires Internationales":
        tab_secteur[0] += [row["150. 1erEmploiLapsPourTrouverApresDiplomeMois"]]
    elif row["263. Ecole_4emeAnneeSecteur"] == "Affaires Publiques":
        tab_secteur[1] += [row["150. 1erEmploiLapsPourTrouverApresDiplomeMois"]]
    elif row["263. Ecole_4emeAnneeSecteur"] == "Communication":
        tab_secteur[2] += [row["150. 1erEmploiLapsPourTrouverApresDiplomeMois"]]
    elif row["263. Ecole_4emeAnneeSecteur"] == "Territoires":
        tab_secteur[3] += [row["150. 1erEmploiLapsPourTrouverApresDiplomeMois"]]
        
moyenne_total = round(stat.mean(tab), 1)
mediane_total = round(stat.median(tab), 1)

moyenne_AI = round(stat.mean(tab_secteur[0]), 1)
moyenne_AP = round(stat.mean(tab_secteur[1]), 1)
moyenne_COM = round(stat.mean(tab_secteur[2]), 1)
moyenne_TER = round(stat.mean(tab_secteur[3]), 1)

mediane_AI = round(stat.median(tab_secteur[0]), 1)
mediane_AP = round(stat.median(tab_secteur[1]), 1)
mediane_COM = round(stat.median(tab_secteur[2]), 1)
mediane_TER = round(stat.median(tab_secteur[3]), 1)

print(f"moy total : {moyenne_total}; mediane total : {mediane_total};  Moyenne :  AI : {moyenne_AI}; AP : {moyenne_AP}; COM : {moyenne_COM}; TER : {moyenne_TER};  Mediane :  AI : {mediane_AI}; AP : {mediane_AP}; COM : {mediane_COM}; TER : {mediane_TER};  ")
print(tab_secteur)

moy total : 1.0; mediane total : 0.0;  Moyenne :  AI : 1.1; AP : 0.7; COM : 0.6; TER : 1.1;  Mediane :  AI : 0.0; AP : 0.0; COM : 0.0; TER : 0.0;  
[[0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 1.0, 0.0, 0.0, 3.0, 0.0, 7.0, 0.0, 1.0, 0.0, 2.0, 0.0, 2.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 7.0, 0.0, 4.0, 5.0, 2.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 5.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 2.0, 0.0, 0.0, 0.0, 4.0, 0.0], [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 6.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0, 0.0, 4.0]]


In [52]:
# Graphique temps avant 1er job après diplôme (mediane et moyenne)

donnees = {
    "mesure" : ["MOYENNE", "MEDIANE"],
    "total" : [moyenne_total, mediane_total],
    "Affaires Internationales" : [moyenne_AI, mediane_AI],
    "Affaires Publiques" : [moyenne_AP, mediane_AP],
    "Communication" : [moyenne_COM, mediane_COM],
    "Territoires" : [moyenne_TER, mediane_TER]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,mesure,total,Affaires Internationales,Affaires Publiques,Communication,Territoires
0,MOYENNE,1.0,1.1,0.7,0.6,1.1
1,MEDIANE,0.0,0.0,0.0,0.0,0.0


In [53]:
# Modification fichier excel

  
sheet_name = "graph_7-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [54]:
# Calcul raison difficultés dans la recherche d'emploi

raison = ["Autre", "Aucune difficulté", "Mobilité géographique limitée", "Méconnaissance des débouchés possibles pour ma formation",
          "Mauvaise maîtrise des techniques de recherche d'emploi (lettre de motivation, CV, entretien d'embauche...)",
          "Manque d'expérience professionnelle", "Difficulté à mettre en valeur mes compétences", 
          "Formation mal, ou pas, reconnue par les employeurs", "Formation inadaptée au marché de l'emploi",
          "Peu d'offres d'emploi", "Salaire proposé insuffisant", "Démarches administratives"]

tab_raison = [0] * len(raison)

nbr_personne = 0

for index, row in D.iterrows():
    if isinstance(row["119. RechercheEmploiDifficultes"], float) and np.isnan(row["119. RechercheEmploiDifficultes"]):
        continue
    buf = row["119. RechercheEmploiDifficultes"].split(";")
    for w in buf:
        tab_raison[raison.index(w)] += 1
    nbr_personne += 1
        
print(tab_raison)
print(nbr_personne)


[3, 0, 1, 6, 2, 18, 5, 2, 4, 10, 6, 1]
25


In [55]:
# Graphique raison difficultés dans la recherche d'emploi
raison[0] = "Autres"
donnees = dict(zip(raison, tab_raison))
sortie = pandas.DataFrame([donnees])

sortie["Nombre personne"] = nbr_personne

sortie = sortie.loc[:, sortie.iloc[0] != 0]

sortie.head()

,Autres,Mobilité géographique limitée,Méconnaissance des débouchés possibles pour ma formation,"Mauvaise maîtrise des techniques de recherche d'emploi (lettre de motivation, CV, entretien d'embauche...)",Manque d'expérience professionnelle,Difficulté à mettre en valeur mes compétences,"Formation mal, ou pas, reconnue par les employeurs",Formation inadaptée au marché de l'emploi,Peu d'offres d'emploi,Salaire proposé insuffisant,Démarches administratives,Nombre personne
0,3,1,6,2,18,5,2,4,10,6,1,25


In [56]:
# Modification fichier excel

  
sheet_name = "graph_8-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [57]:
# Calcul moyens de recherche

moyen = [
"Autre",
"Service emploi de votre école",
"Forum de votre école",
"Apprentissage (embauche dans l'entreprise d'accueil du contrat)",
"Stage de fin d'études",
"Stage année césure, année professionnalisante",
"Réseau des anciens élèves",
"Réseaux sociaux professionnels (LinkedIn, ...)",
"Sites internet spécialisés dans l'emploi (APEC, ....)",
"Sites internet d'entreprises",
"Candidature spontanée",
"Chasseur de têtes",
"Relations personnelles",
"Concours",
"J'ai créé / repris une entreprise"
]

tab_moyen = [0] * len(moyen)

nbr_personne = 0

for index, row in D.iterrows():
    if isinstance(row["80. EmploiCommentTrouve"], float) and np.isnan(row["80. EmploiCommentTrouve"]):
        continue
    buf = row["80. EmploiCommentTrouve"].split(";")
    for w in buf:
        tab_moyen[moyen.index(w)] += 1
    nbr_personne += 1
        
total_moyen = sum(tab_moyen)

print(tab_moyen)
print(total_moyen)
print(nbr_personne)

pourcentage_moyen = [0] * len(moyen)

# Pourcentage moyen de recherche
for i in range(len(tab_moyen)):
    pourcentage_moyen[i] = round(tab_moyen[i] / total_moyen, 3)

print(pourcentage_moyen)


[5, 2, 0, 7, 30, 1, 1, 18, 11, 15, 11, 0, 13, 4, 0]
118
118
[0.042, 0.017, 0.0, 0.059, 0.254, 0.008, 0.008, 0.153, 0.093, 0.127, 0.093, 0.0, 0.11, 0.034, 0.0]


In [58]:
# Graphique raison difficultés dans la recherche d'emploi
moyen[0] = "Autres"
donnees = dict(zip(moyen, pourcentage_moyen))
sortie = pandas.DataFrame([donnees])

sortie["Nombre personne"] = nbr_personne

sortie = sortie.loc[:, sortie.iloc[0] != 0.0]

sortie.head()

,Autres,Service emploi de votre école,Apprentissage (embauche dans l'entreprise d'accueil du contrat),Stage de fin d'études,"Stage année césure, année professionnalisante",Réseau des anciens élèves,"Réseaux sociaux professionnels (LinkedIn, ...)","Sites internet spécialisés dans l'emploi (APEC, ....)",Sites internet d'entreprises,Candidature spontanée,Relations personnelles,Concours,Nombre personne
0,0.042,0.017,0.059,0.254,0.008,0.008,0.153,0.093,0.127,0.093,0.11,0.034,118


In [59]:
# Modification fichier excel

  
sheet_name = "graph_8-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [60]:
# Calcul type employeur

asso_ong = 0
secteur_prive = 0
secteur_public = 0

for index, row in D.iterrows():
    if row["42. EmploiEntrepriseStatut"] == "Association / ONG":
        asso_ong += 1
    elif row["42. EmploiEntrepriseStatut"] == "Secteur privé":
        secteur_prive += 1
    elif row["42. EmploiEntrepriseStatut"] == "Secteur public":
        secteur_public += 1

total_emp = asso_ong + secteur_prive + secteur_public

print(f"total : {total_emp}; asso/ong : {asso_ong}; secteur privé : {secteur_prive}; secteur public : {secteur_public}")

# Pourcentage type employeur
        
pourcentage_asso_ong = round(asso_ong / total_emp, 3)
pourcentage_secteur_prive = round(secteur_prive / total_emp, 3)
pourcentage_secteur_public = round(secteur_public / total_emp, 3)

print(f"% : ; asso/ong : {pourcentage_asso_ong}; secteur privé : {pourcentage_secteur_prive}; secteur public : {pourcentage_secteur_public}")


total : 118; asso/ong : 18; secteur privé : 54; secteur public : 46
% : ; asso/ong : 0.153; secteur privé : 0.458; secteur public : 0.39


In [61]:
# Graphique type employeur

donnees = {
    "annee" : [2021, 2022, 2023, "2023 arrondis"],
    "Association / ONG" : [0.212, 0.185, pourcentage_asso_ong, round(pourcentage_asso_ong * 100, 1)],
    "Secteur privé" : [0.474, 0.504,  pourcentage_secteur_prive, round(pourcentage_secteur_prive * 100, 1)],
    "Secteur public" : [0.314, 0.311, pourcentage_secteur_public, round(pourcentage_secteur_public * 100, 1)],
    "nombre personne" : [0, 0, total_emp, total_emp]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,annee,Association / ONG,Secteur privé,Secteur public,nombre personne
0,2021,0.212,0.474,0.314,0
1,2022,0.185,0.504,0.311,0
2,2023,0.153,0.458,0.390,118
3,2023 arrondis,15.300,45.800,39.000,118


In [62]:
# Modification fichier excel

  
sheet_name = "graph_9-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [63]:
donnees = {
    "Est en relation à l'étranger" : [pourcentage_apparition_DF(["66. Option_EmploiInternational"], ["Oui"], [D])]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,Est en relation à l'étranger
0,39.0


In [64]:
# Modification fichier excel

  
sheet_name = "graph_9-3"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [65]:
# Calcul taille entreprise

taille_entreprise = ["De 1 à 9", "De 10 à 19",
                 "De 20 à 49", "De 50 à 249",
                 "De 250 à 4 999", "5 000 ou plus"]

tab_taille = [0] * len(taille_entreprise)

for index, row in D.iterrows():
    if isinstance(row["49. EmploiEntrepriseTaille"], float) and np.isnan(row["49. EmploiEntrepriseTaille"]):
        continue
    buf = row["49. EmploiEntrepriseTaille"].split(";")
    for w in buf:
        tab_taille[taille_entreprise.index(w)] += 1

total_taille = sum(tab_taille)

print(tab_taille)
print(total_taille)

# Pourcentage taille entreprise

pourcentage_taille = [0] * len(taille_entreprise)

for i in range(len(taille_entreprise)):
    pourcentage_taille[i] = round(tab_taille[i] / total_taille * 100, 1)
    
print(pourcentage_taille)


[17, 9, 15, 23, 26, 23]
113
[15.0, 8.0, 13.3, 20.4, 23.0, 20.4]


In [66]:
# Graphique taille entreprise

TPE = pourcentage_taille[0]
PME = pourcentage_taille[1] + pourcentage_taille[2] + pourcentage_taille[3]
ETI = pourcentage_taille[4]
GE = pourcentage_taille[5]

donnees = {
    "TPE" : [TPE],
    "PME" : [PME],
    "ETI" : [ETI],
    "GE" : [GE]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,TPE,PME,ETI,GE
0,15.0,41.7,23.0,20.4


In [67]:
# Modification fichier excel

  
sheet_name = "graph_9-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [68]:
# Calcul domaine d'activité

categorie = [
    "Autre",
    "Agriculture, sylviculture et pêche",
    "Industries extractives",
    "Industrie agroalimentaire",
    "Fabrication de textiles - Industries de l'habillement, du cuir et de la chaussure",
    "Industrie chimique",
    "Industrie pharmaceutique",
    "Fabrication de produits en caoutchouc et en plastique, autres produits minéraux non métalliques",
    "Métallurgie et fabrication de produits métalliques, hors machines et équipements",
    "Industrie automobile, aéronautique, navale, ferroviaire",
    "Industrie des Technologies de l'Information et de la Communication (TIC)",
    "Autres industries (bois, imprimerie, fabrication, réparation et installation de machines et d'équipements, ...)",
    "Énergie (Production et distribution d'électricité, de gaz, de vapeur et d'air conditionné)",
    "Production et distribution d'eau assainissement, gestion des déchets et dépollution",
    "Construction, BTP",
    "Commerce",
    "Transports (Services)",
    "Hébergement et restauration",
    "Édition, audiovisuel et diffusion",
    "Télécommunications",
    "Activités informatiques et services d'information (TIC Services)",
    "Activités financières et d'assurance",
    "Activités immobilières",
    "Activités juridiques, comptables, de gestion, d'architecture, de contrôle et d'analyses techniques",
    "Société de conseil ou d'ingénierie - Bureaux d'études indépendants",
    "Recherche-développement scientifique",
    "Communication - Publicité - Marketing - Autres activités spécialisées, scientifiques et techniques",
    "Tourisme",
    "Administration d'État - Collectivités territoriales - Hospitalière",
    "Enseignement - Recherche",
    "Santé humaine et action sociale",
    "Arts, spectacles et activités récréatives"
]

tab_categorie = [0] * len(categorie)
tab_categorie_H = [0] * len(categorie)
tab_categorie_F = [0] * len(categorie)

for index, row in D.iterrows():
    if isinstance(row["55. EmploiSecteur"], float) and np.isnan(row["55. EmploiSecteur"]):
        continue
    tab_categorie[categorie.index(row["55. EmploiSecteur"])] += 1
    if row["4. IdentiteSexeVerifie"] == "Femme":
        tab_categorie_F[categorie.index(row["55. EmploiSecteur"])] += 1
    elif row["4. IdentiteSexeVerifie"] == "Homme":
        tab_categorie_H[categorie.index(row["55. EmploiSecteur"])] += 1

total_categorie = sum(tab_categorie)
total_categorie_F = sum(tab_categorie_F)
total_categorie_H = sum(tab_categorie_H)

    
print(tab_categorie)
print(total_categorie)
print(f"F : {total_categorie_F}")
print(f"H : {total_categorie_H}")

# Pourcentage domaine d'activité

pourcentage_categorie = [0] * len(categorie)
pourcentage_categorie_F = [0] * len(categorie)
pourcentage_categorie_H = [0] * len(categorie)

for i in range(len(categorie)):
    pourcentage_categorie[i] = round(tab_categorie[i] / total_categorie, 3)
    pourcentage_categorie_F[i] = round(tab_categorie_F[i] / total_categorie_F, 3)
    pourcentage_categorie_H[i] = round(tab_categorie_H[i] / total_categorie_H, 3)

print(pourcentage_categorie)
print(f"F : {pourcentage_categorie_F}")
print(f"H : {pourcentage_categorie_H}")


[20, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 1, 1, 1, 3, 0, 4, 2, 1, 0, 17, 1, 5, 2, 27, 8, 7, 11]
117
F : 84
H : 33
[0.171, 0.009, 0.0, 0.017, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009, 0.0, 0.017, 0.0, 0.0, 0.009, 0.009, 0.009, 0.026, 0.0, 0.034, 0.017, 0.009, 0.0, 0.145, 0.009, 0.043, 0.017, 0.231, 0.068, 0.06, 0.094]
F : [0.143, 0.012, 0.0, 0.012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.012, 0.0, 0.012, 0.0, 0.0, 0.012, 0.012, 0.012, 0.024, 0.0, 0.036, 0.0, 0.012, 0.0, 0.143, 0.012, 0.048, 0.012, 0.226, 0.071, 0.071, 0.119]
H : [0.242, 0.0, 0.0, 0.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03, 0.0, 0.03, 0.061, 0.0, 0.0, 0.152, 0.0, 0.03, 0.03, 0.242, 0.061, 0.03, 0.03]


In [69]:
# Graphique domaine d'activité

categorie[0] = "Autres"

donnees = dict(zip(categorie, pourcentage_categorie))
donnees_F = dict(zip(categorie, pourcentage_categorie_F))
donnees_H = dict(zip(categorie, pourcentage_categorie_H))

# Création des DataFrames à partir des dictionnaires
sortie = pandas.DataFrame([donnees])
df_H = pandas.DataFrame([donnees_H])
df_F = pandas.DataFrame([donnees_F])

# Ajout d'une colonne pour différencier les groupes
sortie.insert(0, 'Groupe', 'total') 
df_H.insert(0, 'Groupe', 'Hommes')
df_F.insert(0, 'Groupe', 'Femmes')

# Combinaison des DataFrames
sortie = pandas.concat([sortie, df_H, df_F], ignore_index=True)

sortie["Nombre personne"] = [total_categorie, total_categorie_H, total_categorie_F]

sortie = sortie.loc[:, sortie.iloc[0] != 0.0] # Les colonnes avec un pourcentage de 0 sont enlevées 

sortie.head()

,Groupe,Autres,"Agriculture, sylviculture et pêche",Industrie agroalimentaire,Industrie des Technologies de l'Information et de la Communication (TIC),"Énergie (Production et distribution d'électricité, de gaz, de vapeur et d'air conditionné)",Commerce,Transports (Services),Hébergement et restauration,"Édition, audiovisuel et diffusion",...,Activités immobilières,Société de conseil ou d'ingénierie - Bureaux d'études indépendants,Recherche-développement scientifique,"Communication - Publicité - Marketing - Autres activités spécialisées, scientifiques et techniques",Tourisme,Administration d'État - Collectivités territoriales - Hospitalière,Enseignement - Recherche,Santé humaine et action sociale,"Arts, spectacles et activités récréatives",Nombre personne
0,total,0.171,0.009,0.017,0.009,0.017,0.009,0.009,0.009,0.026,...,0.009,0.145,0.009,0.043,0.017,0.231,0.068,0.060,0.094,117
1,Hommes,0.242,0.000,0.030,0.000,0.030,0.000,0.000,0.000,0.030,...,0.000,0.152,0.000,0.030,0.030,0.242,0.061,0.030,0.030,33
2,Femmes,0.143,0.012,0.012,0.012,0.012,0.012,0.012,0.012,0.024,...,0.012,0.143,0.012,0.048,0.012,0.226,0.071,0.071,0.119,84


In [70]:
# Modification fichier excel

  
sheet_name = "graph_10-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [71]:
# domaine d'activité autre

tab_categorie_autre = []

for index, row in D.iterrows():
    if isinstance(row["56. EmploiSecteurAutre"], float) and np.isnan(row["56. EmploiSecteurAutre"]):
        continue
    tab_categorie_autre += [row["56. EmploiSecteurAutre"]]

print(tab_categorie_autre)

['Cultuel', 'Défense militaire', 'Ecologie', 'Média télévisuelle', 'Economie - monnaies locales', 'Politique', 'Député ', 'Relations internationales et institutionnelles', "Association de protection de l'environnement", 'Musée ', 'ONG environmentale', 'Autres activités informatiques ', 'Agence de presse, média ', 'Association', 'Humanitaire / associatif ', 'Économie sociale et solidaire - information et communication sur les sujets européens', 'Sport', 'Presse']


In [72]:
# Calcul contrat

contrat = [
    "Contrat local",
    "Contrat d'expatriation",
    "CDI - y compris de la fonction publique",
    "CDIC",
    "Titulaire ou stagiaire de la fonction publique",
    "CDD - y compris de la fonction publique",
    "CTT",
    "Autre"
]

tab_contrat = [0] * len(contrat)

for index, row in D.iterrows():
    if isinstance(row["30. EmploiContrat"], float) and np.isnan(row["30. EmploiContrat"]):
        continue
    tab_contrat[contrat.index(row["30. EmploiContrat"])] += 1

total_contrat = sum(tab_contrat)

print(tab_contrat)
print(total_contrat)

# Pourcentage contrat

pourcentage_contrat = [0] * len(contrat)

for i in range(len(contrat)):
    pourcentage_contrat[i] = round(tab_contrat[i] / total_contrat, 3)

print(pourcentage_contrat)

[11, 2, 35, 0, 3, 61, 0, 4]
116
[0.095, 0.017, 0.302, 0.0, 0.026, 0.526, 0.0, 0.034]


In [73]:
# Graphique contrat

CDI = pourcentage_contrat[2] + pourcentage_contrat[3]
CDD = pourcentage_contrat[5]
autre = pourcentage_contrat[0] + pourcentage_contrat[1] + pourcentage_contrat[4] + pourcentage_contrat[6] + pourcentage_contrat[7] 

donnees = {
    "Année" : [2021, 2022, 2023],
    "CDI" : [0.529, 0.477, CDI],
    "CDD" : [0.307, 0.392, CDD],
    "Autres" : [0.163, 0.131, autre]
}

sortie = pandas.DataFrame(donnees)

sortie["Nombre de personne"] = [0, 0, total_contrat]

sortie.head()

,Année,CDI,CDD,Autres,Nombre de personne
0,2021,0.529,0.307,0.163,0
1,2022,0.477,0.392,0.131,0
2,2023,0.302,0.526,0.172,116


In [74]:
# Modification fichier excel

sheet_name = "graph_11-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [75]:
# Calcul durée contrat et plein temps

moins_6_mois = 0
plus_6_mois = 0
temps_plein = 0
temps_partiel = 0

for index, row in D.iterrows():
    if isinstance(row["32. EmploiContratFranceDuree"], float) and np.isnan(row["32. EmploiContratFranceDuree"]):
        continue
    if row["32. EmploiContratFranceDuree"] >= 6:
        plus_6_mois += 1
    else:
        moins_6_mois += 1
    
total_duree = plus_6_mois + moins_6_mois
    
for index, row in D.iterrows(): 
    if isinstance(row["33. EmploiTempsPleinPartiel"], float) and np.isnan(row["33. EmploiTempsPleinPartiel"]):
        continue
    if row["33. EmploiTempsPleinPartiel"] == "À temps plein":
        temps_plein += 1
    elif row["33. EmploiTempsPleinPartiel"] == "À temps partiel":
        temps_partiel += 1
        
total_temps = temps_plein + temps_partiel

print(f"total duree : {total_duree}")
print(f"total temps : {total_temps}")
print(f"-6 mois : {moins_6_mois}; +6 mois : {plus_6_mois}; temps plein : {temps_plein}; :temps partiel : {temps_partiel}")
        
# Pourcentage durée contrat et plein temps

pourcentage_plus_6_mois = int(round(plus_6_mois / total_duree * 100, 0))
pourcentage_temps_plein = int(round(temps_plein / total_temps * 100, 0))


total duree : 59
total temps : 116
-6 mois : 10; +6 mois : 49; temps plein : 110; :temps partiel : 6


In [76]:
# Graphique durée contrat et plein temps

donnees = {
    "plus de 6 mois" : [pourcentage_plus_6_mois],
    "temps plein" : [pourcentage_temps_plein]
}

sortie = pandas.DataFrame(donnees)

sortie.head()

,plus de 6 mois,temps plein
0,83,95


In [77]:
# Modification fichier excel

sheet_name = "graph_11-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [78]:
# Calcul contrat selon le secteur de 4ème année


contrat = [
    "Contrat local",
    "Contrat d'expatriation",
    "CDI - y compris de la fonction publique",
    "CDIC",
    "Titulaire ou stagiaire de la fonction publique",
    "CDD - y compris de la fonction publique",
    "CTT",
    "Autre"
]

tab_contrat_AI = [0] * len(contrat)
tab_contrat_AP = [0] * len(contrat)
tab_contrat_COM = [0] * len(contrat)
tab_contrat_TER = [0] * len(contrat)

for index, row in D.iterrows():
    if isinstance(row["30. EmploiContrat"], float) and np.isnan(row["30. EmploiContrat"]):
        continue
    if row["263. Ecole_4emeAnneeSecteur"] == "Affaires Internationales":
        tab_contrat_AI[contrat.index(row["30. EmploiContrat"])] += 1 
    elif row["263. Ecole_4emeAnneeSecteur"] == "Affaires Publiques":
        tab_contrat_AP[contrat.index(row["30. EmploiContrat"])] += 1
    elif row["263. Ecole_4emeAnneeSecteur"] == "Communication":
        tab_contrat_COM[contrat.index(row["30. EmploiContrat"])] += 1
    elif row["263. Ecole_4emeAnneeSecteur"] == "Territoires":
        tab_contrat_TER[contrat.index(row["30. EmploiContrat"])] += 1

total_contrat_AI = sum(tab_contrat_AI) 
total_contrat_AP = sum(tab_contrat_AP)
total_contrat_COM = sum(tab_contrat_COM)
total_contrat_TER = sum(tab_contrat_TER)

print(f"AI : {tab_contrat_AI}")
print(f"AP : {tab_contrat_AP}")
print(f"COM : {tab_contrat_COM}")
print(f"TER : {tab_contrat_TER}")

print("total :", total_contrat)

# Pourcentage contrat selon le secteur de 4ème année

pourcentage_contrat_AI = [0] * len(contrat)
pourcentage_contrat_AP = [0] * len(contrat)
pourcentage_contrat_COM = [0] * len(contrat)
pourcentage_contrat_TER = [0] * len(contrat)

for i in range(len(contrat)):
    pourcentage_contrat_AI[i] = round(tab_contrat_AI[i] / total_contrat_AI, 3)
    pourcentage_contrat_AP[i] = round(tab_contrat_AP[i] / total_contrat_AP, 3)
    pourcentage_contrat_COM[i] = round(tab_contrat_COM[i] / total_contrat_COM, 3)
    pourcentage_contrat_TER[i] = round(tab_contrat_TER[i] / total_contrat_TER, 3)

print(f"AI : {pourcentage_contrat_AI}")
print(f"AP : {pourcentage_contrat_AP}")
print(f"COM : {pourcentage_contrat_COM}")
print(f"TER : {pourcentage_contrat_TER}")

AI : [4, 2, 13, 0, 0, 23, 0, 0]
AP : [2, 0, 10, 0, 2, 11, 0, 2]
COM : [3, 0, 9, 0, 0, 13, 0, 2]
TER : [1, 0, 3, 0, 1, 7, 0, 0]
total : 116
AI : [0.095, 0.048, 0.31, 0.0, 0.0, 0.548, 0.0, 0.0]
AP : [0.074, 0.0, 0.37, 0.0, 0.074, 0.407, 0.0, 0.074]
COM : [0.111, 0.0, 0.333, 0.0, 0.0, 0.481, 0.0, 0.074]
TER : [0.083, 0.0, 0.25, 0.0, 0.083, 0.583, 0.0, 0.0]


In [79]:
# Graphique contrat selon le secteur de 4ème année

contrat[len(contrat)-1] = "Autres"

donnees = {
    "etiquette" : contrat,
    "Affaires Internationales" : pourcentage_contrat_AI,
    "Affaires Publiques" : pourcentage_contrat_AP,
    "Communication" : pourcentage_contrat_COM,
    "Territoires" : pourcentage_contrat_TER
}

sortie = pandas.DataFrame(donnees)

sortie.head(10)

,etiquette,Affaires Internationales,Affaires Publiques,Communication,Territoires
0,Contrat local,0.095,0.074,0.111,0.083
1,Contrat d'expatriation,0.048,0.000,0.000,0.000
2,CDI - y compris de la fonction publique,0.310,0.370,0.333,0.250
3,CDIC,0.000,0.000,0.000,0.000
4,Titulaire ou stagiaire de la fonction publique,0.000,0.074,0.000,0.083
5,CDD - y compris de la fonction publique,0.548,0.407,0.481,0.583
6,CTT,0.000,0.000,0.000,0.000
7,Autres,0.000,0.074,0.074,0.000


In [80]:
# Modification fichier excel

sheet_name = "graph_11-3"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [81]:
# Calcul rémunération 

rang = ["Moins de 20 000€", 
        "De 20 000€ à 24 999€", 
        "De 25 000€ à 29 999€", 
        "De 30 000€ à 34 999€", 
        "De 35 000€ à 39 999€", 
        "40 000€ et plus"]

tab_salaire = []
rang_salaire = [0] * len(rang)

for index, row in D.iterrows():
    if isinstance(row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"], float) and np.isnan(row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"]):
        continue
    if row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 20000:
        rang_salaire[0] += 1
    elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 20000 and row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 25000:
        rang_salaire[1] += 1
    elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 25000 and row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 30000:
        rang_salaire[2] += 1
    elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 30000 and row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 35000:
        rang_salaire[3] += 1
    elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 35000 and row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 40000:
        rang_salaire[4] += 1
    elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 40000:
        rang_salaire[5] += 1
    else:
        print("Erreur : rang salaire")

    tab_salaire += [row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"]]
    total_rang_salaire = sum(rang_salaire)

print(tab_salaire)
print(rang_salaire)
print(total_rang_salaire)

moyenne_salaire = round(stat.mean(tab_salaire), 0)
mediane_salaire = stat.median(tab_salaire)

print("moy : ", moyenne_salaire)
print("mediane : ", mediane_salaire)

# Pourcentage rang salaire

pourcentage_salaire = [0] * len(rang)

for i in range(len(rang)):
    pourcentage_salaire[i] = round(rang_salaire[i] / total_rang_salaire, 3)
    
print(pourcentage_salaire)



[33000.0, 37450.0, 34500.0, 19200.0, 24000.0, 25200.0, 30500.0, 30000.0, 35000.0, 30000.0, 13200.0, 27000.0, 28800.0, 81500.0, 27700.0, 33000.0, 41000.0, 24000.0, 43500.0, 30000.0, 33200.0, 11724.0, 25220.0, 40300.0, 31928.0, 19200.0, 5990.0, 40000.0, 29105.0, 35500.0, 41100.0, 47000.0, 34500.0, 27800.0, 39002.0, 42000.0, 30000.0, 52000.0, 45500.0, 28680.0, 29000.0, 37970.0, 36500.0, 34000.0, 22000.0, 39000.0, 24700.0, 34800.0, 36000.0, 28400.0, 29700.0, 26700.0, 19220.0, 66800.0, 48000.0, 26160.0, 27028.0, 33600.0, 33000.0, 32300.0, 26000.0, 31000.0, 33000.0, 36800.0, 27600.0, 48000.0, 29300.0, 21700.0, 31000.0, 29600.0, 15600.0, 29100.0, 31300.0, 31200.0, 37000.0, 30000.0, 25200.0, 36000.0, 33762.0, 45500.0, 36000.0, 40767.0, 31000.0, 44000.0, 45000.0, 28000.0, 32000.0, 31959.0, 38000.0, 16200.0, 32400.0, 21600.0, 33234.0, 66000.0, 39000.0, 34000.0, 36000.0, 33000.0, 30800.0, 22200.0, 24450.0, 35000.0, 44000.0, 32500.0, 39520.0, 33000.0, 10800.0]
[9, 8, 21, 33, 17, 19]
107
moy :  328

In [82]:
# Graphique rang salaire


donnees = {
    "rang" : rang + ["Nombre personne"],
    2021 : [0.021, 0.125, 0.139, 0.229, 0.188, 0.299, 0],
    2022 : [0.008, 0.083, 0.240, 0.231, 0.157, 0.281, 0],
    2023 : pourcentage_salaire + [total_rang_salaire]
}
sortie = pandas.DataFrame(donnees)

 

sortie.head(10)

,rang,2021,2022,2023
0,Moins de 20 000€,0.021,0.008,0.084
1,De 20 000€ à 24 999€,0.125,0.083,0.075
2,De 25 000€ à 29 999€,0.139,0.240,0.196
3,De 30 000€ à 34 999€,0.229,0.231,0.308
4,De 35 000€ à 39 999€,0.188,0.157,0.159
5,40 000€ et plus,0.299,0.281,0.178
6,Nombre personne,0.000,0.000,107.000


In [83]:
# Modification fichier excel

sheet_name = "graph_12-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [84]:
# Graphique rémunération

donnees = {
    "Année" : [2021, 2022, 2023],
    "Salaire brut annuel moyen (primes comprises)" : [35854, 35394, moyenne_salaire],
    "Salaire brut annuel médian (primes comprises)" : [34800, 32500, mediane_salaire],
}
print(donnees)

sortie = pandas.DataFrame(donnees)

sortie.head(10)

{'Année': [2021, 2022, 2023], 'Salaire brut annuel moyen (primes comprises)': [35854, 35394, 32825.0], 'Salaire brut annuel médian (primes comprises)': [34800, 32500, 32300.0]}


,Année,Salaire brut annuel moyen (primes comprises),Salaire brut annuel médian (primes comprises)
0,2021,35854.0,34800.0
1,2022,35394.0,32500.0
2,2023,32825.0,32300.0


In [85]:
# Modification fichier excel

sheet_name = "graph_12-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [86]:
# Calcul rémunération 

rang = ["Moins de 20 000€", 
        "De 20 000€ à 24 999€", 
        "De 25 000€ à 29 999€", 
        "De 30 000€ à 34 999€", 
        "De 35 000€ à 39 999€", 
        "40 000€ et plus"]

secteur = ["Affaires Internationales",
           "Affaires Publiques",
           "Communication",
           "Territoires"]

tab_salaire = []
rang_salaire = [[0 for _ in range(len(rang))] for _ in range(len(secteur))] # Tableau par secteur puis par rang
moyenne_salaire_secteur = [0] * len(secteur)
tab_salaire_secteur = [[] for _ in range(len(secteur))] 

for i in range(len(secteur)):   
    for index, row in D.iterrows():
        if row["263. Ecole_4emeAnneeSecteur"] != secteur[i]:
            continue
        if isinstance(row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"], float) and np.isnan(row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"]):
            continue
        if row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 20000:
            rang_salaire[i][0] += 1  
        elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 20000 and row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 25000:
            rang_salaire[i][1] += 1
        elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 25000 and row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 30000:
            rang_salaire[i][2] += 1
        elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 30000 and row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 35000:
            rang_salaire[i][3] += 1
        elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 35000 and row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] < 40000:
            rang_salaire[i][4] += 1
        elif row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"] >= 40000:
            rang_salaire[i][5] += 1
        else:
            print("Erreur : rang salaire")
        tab_salaire_secteur[i] += [row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"]]

total_rang_salaire = [0] * 4
    
for i in range(len(secteur)):
    total_rang_salaire[i] = sum(rang_salaire[i])
    moyenne_salaire_secteur[i] = round(stat.mean(tab_salaire_secteur[i]), 0)

print(moyenne_salaire_secteur)

print(rang_salaire)
print(total_rang_salaire)

# Pourcentage rang salaire

pourcentage_salaire = [[0] * len(rang) for _ in range(len(secteur))] # Tableau par secteur puis par rang

print(rang_salaire)

for s in range(len(secteur)):
    
    for i in range(len(rang)):
        pourcentage_salaire[s][i] = round(rang_salaire[s][i] / total_rang_salaire[s], 3)

print(pourcentage_salaire)



[33061.0, 34973.0, 30364.0, 33174.0]
[[4, 3, 8, 10, 7, 8], [1, 3, 2, 8, 5, 6], [1, 2, 9, 9, 2, 2], [3, 0, 1, 4, 3, 3]]
[40, 25, 25, 14]
[[4, 3, 8, 10, 7, 8], [1, 3, 2, 8, 5, 6], [1, 2, 9, 9, 2, 2], [3, 0, 1, 4, 3, 3]]
[[0.1, 0.075, 0.2, 0.25, 0.175, 0.2], [0.04, 0.12, 0.08, 0.32, 0.2, 0.24], [0.04, 0.08, 0.36, 0.36, 0.08, 0.08], [0.214, 0.0, 0.071, 0.286, 0.214, 0.214]]


In [87]:
# Graphique rémunération

donnees = dict(zip(secteur, pourcentage_salaire))

sortie = pandas.DataFrame(donnees)

sortie.insert(0, "Rang", rang)

nouvelle_ligne = dict(zip(secteur, moyenne_salaire_secteur))
nouvelle_ligne.update({"Rang" : "Moyenne"})

sortie.loc[len(sortie.index)] = nouvelle_ligne

sortie.head(10)

,Rang,Affaires Internationales,Affaires Publiques,Communication,Territoires
0,Moins de 20 000€,0.100,0.04,0.04,0.214
1,De 20 000€ à 24 999€,0.075,0.12,0.08,0.000
2,De 25 000€ à 29 999€,0.200,0.08,0.36,0.071
3,De 30 000€ à 34 999€,0.250,0.32,0.36,0.286
4,De 35 000€ à 39 999€,0.175,0.20,0.08,0.214
5,40 000€ et plus,0.200,0.24,0.08,0.214
6,Moyenne,33061.000,34973.00,30364.00,33174.000


In [88]:
# Modification fichier excel

sheet_name = "graph_13-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [89]:
# Calcul rémunération selon le type d'employeur 

statut = ["Association / ONG",
          "Secteur privé",
          "Secteur public"]

statut_salaire = [[] for _ in range(len(statut))]
print(statut_salaire)

for i in range(len(statut)):
    for index, row in D.iterrows():
        if row["42. EmploiEntrepriseStatut"] != statut[i]:
            continue
        if isinstance(row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"], float) and np.isnan(row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"]):
            continue
        statut_salaire[i] += [row["79. Calcul_EmploiSalaireBrutAnnuelAvecPrimes"]]

print(statut_salaire)

moyenne_statut_salaire = [0] * len(statut)

for i in range(len(statut)):
    moyenne_statut_salaire[i] = round(stat.mean(statut_salaire[i]), 0)

print(moyenne_statut_salaire)




[[], [], []]
[[27700.0, 11724.0, 31928.0, 5990.0, 29000.0, 28400.0, 27028.0, 37000.0, 32000.0, 38000.0, 32400.0, 33000.0, 22200.0, 39520.0, 33000.0, 10800.0], [33000.0, 19200.0, 25200.0, 30500.0, 30000.0, 35000.0, 28800.0, 81500.0, 33000.0, 41000.0, 24000.0, 43500.0, 40300.0, 19200.0, 40000.0, 35500.0, 41100.0, 39002.0, 42000.0, 52000.0, 45500.0, 36500.0, 34000.0, 22000.0, 39000.0, 24700.0, 34800.0, 36000.0, 29700.0, 26700.0, 66800.0, 26000.0, 31000.0, 27600.0, 29100.0, 31300.0, 25200.0, 36000.0, 45500.0, 36000.0, 31000.0, 44000.0, 45000.0, 28000.0, 16200.0, 36000.0, 30800.0, 35000.0, 44000.0, 32500.0], [37450.0, 34500.0, 24000.0, 30000.0, 13200.0, 27000.0, 30000.0, 33200.0, 25220.0, 29105.0, 47000.0, 34500.0, 27800.0, 30000.0, 28680.0, 37970.0, 19220.0, 48000.0, 26160.0, 33600.0, 33000.0, 32300.0, 33000.0, 36800.0, 48000.0, 29300.0, 21700.0, 31000.0, 29600.0, 15600.0, 31200.0, 30000.0, 33762.0, 40767.0, 31959.0, 21600.0, 33234.0, 66000.0, 39000.0, 34000.0, 24450.0]]
[27481.0, 35194.0,

In [90]:
# Graphique moyenne rémunération par type d'employeur

donnees = {
    "statut" : statut, 
    2021 : [30106, 40742, 31752],
    2022 : [27775, 38613, 33988],
    2023 : moyenne_statut_salaire
}

sortie = pandas.DataFrame(donnees)

sortie.head(10)

,statut,2021,2022,2023
0,Association / ONG,30106,27775,27481.0
1,Secteur privé,40742,38613,35194.0
2,Secteur public,31752,33988,32021.0


In [91]:
# Modification fichier excel

sheet_name = "graph_13-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [92]:
# Calcul poursuite d'études / doctorat

type_diplome = [
    "MSc de la CGE - Master of Science",
    "Mastère spécialisé accrédité par la CGE",
    "Master",
    "MBA - Master of Business Administration",
    "Autre"
]

concours = 0
doctorat = 0
diplome = 0

for index, row in D.iterrows():
    if row["15. Situation"] == "En études / En formation":
        if row["140. EtudesType"] == "Préparation d'un concours":
            concours += 1
        elif row["140. EtudesType"] in type_diplome:
            diplome += 1
    if row["15. Situation"] == "En thèse / PhD":
        doctorat += 1
        
print(f"concours : {concours}; doctorat : {doctorat}; diplome : {diplome}")

concours : 12; doctorat : 3; diplome : 30


In [93]:
# Graphique poursuite d'études / doctorat

donnees = {
    "poursuite" : ["Répondants préparent actuellement un concours", "Répondants préparent actuellement un doctorat",
                   "Répondants préparent actuellement un autre diplôme"],
    "2023" : [concours, doctorat, diplome]
}

sortie = pandas.DataFrame(donnees)

sortie.head(10)


,poursuite,2023
0,Répondants préparent actuellement un concours,12
1,Répondants préparent actuellement un doctorat,3
2,Répondants préparent actuellement un autre dip...,30


In [94]:
# Modification fichier excel

sheet_name = "graph_14-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [95]:
# Calcul liste autre diplômes et concours

liste_diplome = []
liste_concours = []

for index, row in D.iterrows():
    if isinstance(row["141. EtudesTypeAutre"], float) and np.isnan(row["141. EtudesTypeAutre"]):
        continue
    liste_diplome += [row["141. EtudesTypeAutre"]]

for index, row in D.iterrows():
    if isinstance(row["142. Option_EtudesTypeConcours"], float) and np.isnan(row["142. Option_EtudesTypeConcours"]):
        continue
    liste_concours += [row["142. Option_EtudesTypeConcours"]]

print(liste_diplome)
print(liste_concours)
    

['Formation professionnelle - Cheffe de projet musique ', 'DLC Arabe oriental/littéral ', 'L3 histoire de l’art ', 'En préparation du CAP cuisine en alternance', 'Formation haute fonction publique hospitalière ', 'Russe langue étrangère', 'auditrice de justice', 'DU DecCapro (pour stage', 'École d’avocat', "Elève avocate - Certificat d'Aptitude à la Profession d'Avocat", 'Ecole de chant', "Cette année était une année de transition où j'ai préparé l'examen pour l'obtention de la licence d'agent sportif et où j'ai en parallèle poursuivi mes études dans le droit et je prépare la rentrée académique prochaine pour laquelle je cherche à intégrer un master juridique en vue de passer l'examen d'accès au CRFPA", 'formation professionnelle ', 'Élève directrice d’hôpital ', 'CAP METIERS DE LA MODE VÊTEMENT FLOU', 'Licence LEA coréen ']
['Agregation externe de Sciences économiques et sociales', 'IRA / INSP', 'Préparation en particulier du concours de Secrétaire des Affaires étrangères ', 'Institut

In [96]:
# Calcul situation

nombre_etude_these = 0 # En poursuite d'études
nombre_reste = 0

for index, row in D.iterrows():
    if isinstance(row["15. Situation"], float) and np.isnan(row["15. Situation"]):
        continue
    if row["15. Situation"] == "En études / En formation":
        nombre_etude_these += 1
    elif row["15. Situation"] == "En thèse / PhD":
        nombre_etude_these += 1
    else:
        nombre_reste += 1

total_activite = nombre_etude_these + nombre_reste 
        
print(f"total : {total_activite}; en poursuite d'études : {nombre_etude_these}; reste : {nombre_reste}")

# Pourcentage situation

pourcentage_etude_these = round(nombre_etude_these / total_activite, 3)

print(f"% etude/these : {pourcentage_etude_these}")




total : 219; en poursuite d'études : 45; reste : 174
% etude/these : 0.205


In [97]:
# Graphique taux poursuite d'étude


donnees = {
    "2021" : [0.155],
    "2022" : [0.179],
    "2023" : [pourcentage_etude_these]
}

sortie = pandas.DataFrame(donnees)

sortie.head(10)

,2021,2022,2023
0,0.155,0.179,0.205


In [98]:
# Modification fichier excel

sheet_name = "graph_14-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [99]:
# Calcul lieu activite à l'étranger

lieu = {}

nb_a_letranger = 0
nb_en_france = 0

nb_actif = 0
nb_volontariat = 0
nb_these = 0

for index, row in D.iterrows():
    # Actif
    if (not isinstance(row["29. EmploiLieuPaysTerritoire"], float) or not np.isnan(row["29. EmploiLieuPaysTerritoire"])):
        if row["29. EmploiLieuPaysTerritoire"] != "France (y compris DOM-COM)":
            if lieu.get(row["29. EmploiLieuPaysTerritoire"]) != None:
                lieu[row["29. EmploiLieuPaysTerritoire"]] += 1
            else:
                lieu[row["29. EmploiLieuPaysTerritoire"]] = 1
            nb_a_letranger += 1
            nb_actif += 1
        else:
            nb_en_france += 1
    # En volontariat
    if (not isinstance(row["104. VolontariatLieuPaysTerritoire"], float) or not np.isnan(row["104. VolontariatLieuPaysTerritoire"])):
        if row["104. VolontariatLieuPaysTerritoire"] != "_France (y compris DOM-COM)":
            if lieu.get(row["104. VolontariatLieuPaysTerritoire"]) != None:
                lieu[row["104. VolontariatLieuPaysTerritoire"]] += 1
            else:
                lieu[row["104. VolontariatLieuPaysTerritoire"]] = 1
            nb_a_letranger += 1
            nb_volontariat += 1
        else:
            nb_en_france += 1
    # En thèse
    if (not isinstance(row["125. TheseLieuPaysTerritoire"], float) or not np.isnan(row["125. TheseLieuPaysTerritoire"])):
        if row["125. TheseLieuPaysTerritoire"] != "France (y compris DOM-COM)":
            if lieu.get(row["125. TheseLieuPaysTerritoire"]) != None:
                lieu[row["125. TheseLieuPaysTerritoire"]] += 1
            else:
                lieu[row["125. TheseLieuPaysTerritoire"]] = 1
            nb_a_letranger += 1
            nb_these += 1
        else:
            nb_en_france += 1
            
total_lieu = nb_a_letranger + nb_en_france

print(lieu)
print(nb_a_letranger)
print(nb_en_france)
print(total_lieu)

# Pourcentage lieu activité à l'étranger

pourcentage_a_letranger = round(nb_a_letranger / total_lieu, 2)

print(pourcentage_a_letranger)


{'Belgique': 3, 'Corée (République de)': 2, 'Suisse': 3, 'Cambodge': 1, 'Royaume-Uni': 2, 'Bénin': 1, 'Ouzbékistan': 1, 'Maroc': 1, 'Chine': 1, 'Éthiopie': 1, 'Japon': 1, 'Irlande': 1, 'Canada': 1, 'Espagne': 2, 'Albanie': 1, 'Argentine': 1, 'Portugal': 1, 'Grèce': 1}
25
113
138
0.18


In [100]:
# Graphique lieu activité à l'étranger

donnees = {
    "etiquette" : ["Taux de diplômés à l'étranger"],
    "nombre à l'étranger" : [nb_a_letranger],
    2021 : [0.12],
    2022 : [0.19],
    2023 : [pourcentage_a_letranger],
    "2023 en %" : [round(pourcentage_a_letranger * 100, 0)],
    "these" : [nb_these],
    "volontariat" : [nb_volontariat],
    "actif" : [nb_actif],
    "total" : [total_lieu]
}

sortie = pandas.DataFrame(donnees)

sortie.head(10)

,etiquette,nombre à l'étranger,2021,2022,2023,2023 en %,these,volontariat,actif,total
0,Taux de diplômés à l'étranger,25,0.12,0.19,0.18,18.0,0,9,16,138


In [101]:
# Modification fichier excel

sheet_name = "graph_15-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [102]:
# Graphique liste lieu activité à l'étranger

donnees = [lieu]

sortie = pandas.DataFrame(donnees)

sortie.insert(0, "Pays", "Nombre de répondants")

sortie["Total"] = nb_a_letranger

sortie.head(10)

,Pays,Belgique,Corée (République de),Suisse,Cambodge,Royaume-Uni,Bénin,Ouzbékistan,Maroc,Chine,Éthiopie,Japon,Irlande,Canada,Espagne,Albanie,Argentine,Portugal,Grèce,Total
0,Nombre de répondants,3,2,3,1,2,1,1,1,1,1,1,1,1,2,1,1,1,1,25


In [103]:
# Modification fichier excel

sheet_name = "graph_15-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [104]:
# Calcul langues étrangères parlées

nb_langue_etranger = 0

langues = {}

for index, row in D.iterrows():
     if row["244. Ecole_EmploiLanguesOuiNon"] == "Oui":
            if isinstance(row["245. Ecole_EmploiLanguesLesquelles"], float) and np.isnan(row["245. Ecole_EmploiLanguesLesquelles"]):
                continue
            buf = row["245. Ecole_EmploiLanguesLesquelles"].split(";")
            for l in buf:
                if langues.get(l) == None:
                    langues.update({l : 1})
                else:
                    langues[l] += 1
            nb_langue_etranger += 1
            
print(langues)
print(nb_langue_etranger)

# Pourcentage langues étrangères parlé

pourcentage_langue_etranger = {}

for l in langues.keys():
    pourcentage_langue_etranger.update({l : round(langues[l] / nb_langue_etranger, 3)})

print(pourcentage_langue_etranger)
            

{'Anglais': 65, 'Espagnol': 17, 'Allemand': 3, 'Chinois': 3, 'Autre': 4, 'Arabe': 4, 'Italien': 1, 'Japonais': 1}
65
{'Anglais': 1.0, 'Espagnol': 0.262, 'Allemand': 0.046, 'Chinois': 0.046, 'Autre': 0.062, 'Arabe': 0.062, 'Italien': 0.015, 'Japonais': 0.015}


In [105]:
# Graphique langues étrangères parlées

donnees = [pourcentage_langue_etranger]

sortie = pandas.DataFrame(donnees)

sortie.insert(0, "Langues", "Pourcentage des langues parlées par les diplômés à l'étranger")

sortie.rename(columns={'Autre': 'Autres'}, inplace=True)

# Nom de la colonne à déplacer
column_to_move = 'Autres'

# Obtenir une liste des colonnes
cols = list(sortie.columns)

# Retirer la colonne et l'ajouter à la fin
cols.append(cols.pop(cols.index(column_to_move)))

# Réorganiser les colonnes du DataFrame
sortie = sortie[cols]

sortie["Nombre total"] = nb_langue_etranger

sortie.head(10)

,Langues,Anglais,Espagnol,Allemand,Chinois,Arabe,Italien,Japonais,Autres,Nombre total
0,Pourcentage des langues parlées par les diplôm...,1.0,0.262,0.046,0.046,0.062,0.015,0.015,0.062,65


In [106]:
# Modification fichier excel

sheet_name = "graph_15-3"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [107]:
# Calcul lieu activité en france

lieu = {}

nb_a_letranger = 0
nb_en_france = 0

for index, row in D.iterrows():
    # Actif
    if (not isinstance(row["28. EmploiLieuRegionEtranger"], float) or not np.isnan(row["28. EmploiLieuRegionEtranger"])):
        if row["28. EmploiLieuRegionEtranger"] != "Étranger":
            if lieu.get(row["28. EmploiLieuRegionEtranger"]) != None:
                lieu[row["28. EmploiLieuRegionEtranger"]] += 1
            else:
                lieu[row["28. EmploiLieuRegionEtranger"]] = 1
    # En volontariat
    if (not isinstance(row["105. VolontariatLieuRegionEtranger"], float) or not np.isnan(row["105. VolontariatLieuRegionEtranger"])):
        if row["105. VolontariatLieuRegionEtranger"] != "Étranger":
            if lieu.get(row["105. VolontariatLieuRegionEtranger"]) != None:
                lieu[row["105. VolontariatLieuRegionEtranger"]] += 1
            else:
                lieu[row["105. VolontariatLieuRegionEtranger"]] = 1
    # En thèse
    if (not isinstance(row["124. TheseLieuRegionEtranger"], float) or not np.isnan(row["124. TheseLieuRegionEtranger"])):
        if row["124. TheseLieuRegionEtranger"] != "Étranger":
            if lieu.get(row["124. TheseLieuRegionEtranger"]) != None:
                lieu[row["124. TheseLieuRegionEtranger"]] += 1
            else:
                lieu[row["124. TheseLieuRegionEtranger"]] = 1

nb_total_lieu = sum(lieu.values())
pourcentage_IDF = round((lieu['Île-de-France'] / nb_total_lieu) * 100, 0)
pourcentage_ARA = round((lieu['Auvergne-Rhône-Alpes'] / nb_total_lieu) * 100, 0)

print(lieu)

{'Île-de-France': 59, 'Auvergne-Rhône-Alpes': 30, 'Bretagne': 4, 'Provence-Alpes-Côte d’Azur': 8, 'Guyane': 1, 'Occitanie': 2, 'Nouvelle-Aquitaine': 2, 'Grand Est': 2, 'Bourgogne-Franche-Comté': 1, 'Pays de la Loire': 2, 'Hauts-de-France': 1, 'La Réunion': 1}


In [108]:
# Graphique lieu activité en france

donnees = [lieu]

sortie = pandas.DataFrame(donnees)

sortie.insert(0, "Département", "Nombre de répondants")

sortie["% Île-de-France"] = pourcentage_IDF
sortie["% Auvergne-Rhône-Alpes"] = pourcentage_ARA

sortie["total"] = nb_total_lieu

sortie.head(10)

,Département,Île-de-France,Auvergne-Rhône-Alpes,Bretagne,Provence-Alpes-Côte d’Azur,Guyane,Occitanie,Nouvelle-Aquitaine,Grand Est,Bourgogne-Franche-Comté,Pays de la Loire,Hauts-de-France,La Réunion,% Île-de-France,% Auvergne-Rhône-Alpes,total
0,Nombre de répondants,59,30,4,8,1,2,2,2,1,2,1,1,52.0,27.0,113


In [109]:
# Modification fichier excel

sheet_name = "graph_16-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [110]:
# Calcul satisfaction formation

critere = ["Contenu de la formation", 
            "Organisation des enseignements", 
            "Adéquation formation universitaire et emploi", 
            "Les études comme moyen d'accès à l'emploi"]

nom_colonne = ["251. Ecole_SatisfactionFormationContenu", 
              "252. Ecole_SatisfactionFormationOrganisation",
              "249. Ecole_SatisfactionFormationAdeqEmploi",
              "250. Ecole_SatisfactionFormationAccesEmploi"]

note = ["À améliorer", "Satisfaisant", "Bien", "Très Bien"]

tab_note = [[0 for _ in range(len(note))] for _ in range(len(critere))]

for i in range(len(critere)):
    for index, row in D.iterrows():
        if isinstance(row[nom_colonne[i]], float) and np.isnan(row[nom_colonne[i]]):
            continue
        tab_note[i][note.index(row[nom_colonne[i]])] += 1

#Total par critère (adéquation formation, ...)
tab_total_critere = [0] * len(critere)

for i in range(len(critere)):
    for j in range(len(note)):
        tab_total_critere[i] += tab_note[i][j]
        
#Total par note (très bien, ...)        
tab_total_note = [0] * len(note)

for i in range(len(note)):
    for j in range(len(critere)):
        tab_total_note[i] += tab_note[j][i]

print("effectif : ", tab_note)
print("total par notes : ", tab_total_note)
print("total par critères :", tab_total_critere)

# Pourcentage satisfaction formation

pourcentage_satisfaction_formation = [[0 for _ in range(len(note))] for _ in range(len(critere))]

for i in range(len(critere)):
    for j in range(len(note)):
        pourcentage_satisfaction_formation[i][j] = round(tab_note[i][j] / tab_total_critere[i], 3)

print("pourcentage : ", pourcentage_satisfaction_formation)

# Pourcentage satisfaction formation total

pourcentage_satisfaction_formation_total = [0] * len(note)

for i in range(len(note)):
    pourcentage_satisfaction_formation_total[i] = round(tab_total_note[i] / sum(tab_total_note), 3)

print("pourcentage total note : ", pourcentage_satisfaction_formation_total)

satisfaction_min = round((sum(pourcentage_satisfaction_formation_total) - pourcentage_satisfaction_formation_total[0]) * 100, 0)
print(satisfaction_min)

effectif :  [[29, 50, 89, 40], [42, 71, 77, 20], [45, 58, 74, 31], [28, 59, 74, 47]]
total par notes :  [144, 238, 314, 138]
total par critères : [208, 210, 208, 208]
pourcentage :  [[0.139, 0.24, 0.428, 0.192], [0.2, 0.338, 0.367, 0.095], [0.216, 0.279, 0.356, 0.149], [0.135, 0.284, 0.356, 0.226]]
pourcentage total note :  [0.173, 0.285, 0.376, 0.165]
83.0


In [111]:
# Graphique satisfaction formation

donnees = dict(zip(critere, pourcentage_satisfaction_formation))

sortie = pandas.DataFrame(donnees)

sortie.insert(0, "Rang", note)

sortie["Satisfaction générale"] = pourcentage_satisfaction_formation_total

sortie["Satisfaction minimum"] = [satisfaction_min] * 4

sortie["Nombre répondants"] = [max(tab_total_critere)] * 4

sortie.head(10)

,Rang,Contenu de la formation,Organisation des enseignements,Adéquation formation universitaire et emploi,Les études comme moyen d'accès à l'emploi,Satisfaction générale,Satisfaction minimum,Nombre répondants
0,À améliorer,0.139,0.200,0.216,0.135,0.173,83.0,210
1,Satisfaisant,0.240,0.338,0.279,0.284,0.285,83.0,210
2,Bien,0.428,0.367,0.356,0.356,0.376,83.0,210
3,Très Bien,0.192,0.095,0.149,0.226,0.165,83.0,210


In [112]:
# Modification fichier excel

sheet_name = "graph_17-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [113]:
# Calcul satisfaction stage

critere = ["Encadrement du stage par les enseignants", 
            "Encadrement du stage par la structure d'accueil", 
            "Compétences acquises", 
            "Déterminant pour l'insertion professionnelle"]

nom_colonne = ["253. Ecole_SatisfactionStageSuiviParEnseig", 
              "254. Ecole_SatisfactionStageSuiviParEntrep",
              "255. Ecole_SatisfactionStageCompetenceAcquis",
              "256. Ecole_SatisfactionStagePourInsertion"]

note = ["À améliorer", "Satisfaisant", "Bien", "Très Bien"]

tab_note = [[0 for _ in range(len(note))] for _ in range(len(critere))]

for i in range(len(critere)):
    for index, row in D.iterrows():
        if isinstance(row[nom_colonne[i]], float) and np.isnan(row[nom_colonne[i]]):
            continue
        tab_note[i][note.index(row[nom_colonne[i]])] += 1

#Total par critère (adéquation formation, ...)
tab_total_critere = [0] * len(critere)

for i in range(len(critere)):
    for j in range(len(note)):
        tab_total_critere[i] += tab_note[i][j]
        
#Total par note (très bien, ...)        
tab_total_note = [0] * len(note)

for i in range(len(note)):
    for j in range(len(critere)):
        tab_total_note[i] += tab_note[j][i]

print("effectif : ", tab_note)
print("total par notes : ", tab_total_note)
print("total par critères :", tab_total_critere)

# Pourcentage satisfaction stage

pourcentage_satisfaction_formation = [[0 for _ in range(len(note))] for _ in range(len(critere))]

for i in range(len(critere)):
    for j in range(len(note)):
        pourcentage_satisfaction_formation[i][j] = round(tab_note[i][j] / tab_total_critere[i], 3)

print("pourcentage : ", pourcentage_satisfaction_formation)

# Pourcentage satisfaction stage total

pourcentage_satisfaction_formation_total = [0] * len(note)

for i in range(len(note)):
    pourcentage_satisfaction_formation_total[i] = round(tab_total_note[i] / sum(tab_total_note), 3)

print("pourcentage total note : ", pourcentage_satisfaction_formation_total)

satisfaction_min = round((sum(pourcentage_satisfaction_formation_total) - pourcentage_satisfaction_formation_total[0]) * 100, 0)
print(satisfaction_min)

effectif :  [[40, 46, 61, 48], [27, 26, 61, 83], [16, 34, 68, 79], [33, 43, 48, 71]]
total par notes :  [116, 149, 238, 281]
total par critères : [195, 197, 197, 195]
pourcentage :  [[0.205, 0.236, 0.313, 0.246], [0.137, 0.132, 0.31, 0.421], [0.081, 0.173, 0.345, 0.401], [0.169, 0.221, 0.246, 0.364]]
pourcentage total note :  [0.148, 0.19, 0.304, 0.358]
85.0


In [114]:
# Graphique satisfaction formation

donnees = dict(zip(critere, pourcentage_satisfaction_formation))

sortie = pandas.DataFrame(donnees)

sortie.insert(0, "Rang", note)

sortie["Satisfaction générale"] = pourcentage_satisfaction_formation_total

sortie["Satisfaction minimum"] = [satisfaction_min] * 4

sortie["Nombre répondants"] = [max(tab_total_critere)] * 4

sortie.head(10)

,Rang,Encadrement du stage par les enseignants,Encadrement du stage par la structure d'accueil,Compétences acquises,Déterminant pour l'insertion professionnelle,Satisfaction générale,Satisfaction minimum,Nombre répondants
0,À améliorer,0.205,0.137,0.081,0.169,0.148,85.0,197
1,Satisfaisant,0.236,0.132,0.173,0.221,0.190,85.0,197
2,Bien,0.313,0.310,0.345,0.246,0.304,85.0,197
3,Très Bien,0.246,0.421,0.401,0.364,0.358,85.0,197


In [115]:
# Modification fichier excel

sheet_name = "graph_17-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [116]:
# Calcul satisfaction emploi

critere = ["Contenu de l'emploi",
            "Perspectives de carrière"]

nom_colonne = ["247. Ecole_EmploiContenuSatisfaction", 
              "248. Ecole_EmploiPerspectivesSatisfaction"]

note = ["À améliorer", "Satisfaisant", "Bien", "Très bien"]

tab_note = [[0 for _ in range(len(note))] for _ in range(len(critere))]

for i in range(len(critere)):
    for index, row in D.iterrows():
        if isinstance(row[nom_colonne[i]], float) and np.isnan(row[nom_colonne[i]]):
            continue
        tab_note[i][note.index(row[nom_colonne[i]])] += 1

#Total par critère (adéquation formation, ...)
tab_total_critere = [0] * len(critere)

for i in range(len(critere)):
    for j in range(len(note)):
        tab_total_critere[i] += tab_note[i][j]
        
#Total par note (très bien, ...)        
tab_total_note = [0] * len(note)

for i in range(len(note)):
    for j in range(len(critere)):
        tab_total_note[i] += tab_note[j][i]

print("effectif : ", tab_note)
print("total par notes : ", tab_total_note)
print("total par critères :", tab_total_critere)

# Pourcentage satisfaction emploi

pourcentage_satisfaction_formation = [[0 for _ in range(len(note))] for _ in range(len(critere))]

for i in range(len(critere)):
    for j in range(len(note)):
        pourcentage_satisfaction_formation[i][j] = round(tab_note[i][j] / tab_total_critere[i], 3)

print("pourcentage : ", pourcentage_satisfaction_formation)

# Pourcentage satisfaction emploi total

pourcentage_satisfaction_formation_total = [0] * len(note)

for i in range(len(note)):
    pourcentage_satisfaction_formation_total[i] = round(tab_total_note[i] / sum(tab_total_note), 3)

print("pourcentage total note : ", pourcentage_satisfaction_formation_total)

satisfaction_min = round((sum(pourcentage_satisfaction_formation_total) - pourcentage_satisfaction_formation_total[0]) * 100, 0)
print(satisfaction_min)

effectif :  [[13, 21, 37, 44], [23, 15, 44, 32]]
total par notes :  [36, 36, 81, 76]
total par critères : [115, 114]
pourcentage :  [[0.113, 0.183, 0.322, 0.383], [0.202, 0.132, 0.386, 0.281]]
pourcentage total note :  [0.157, 0.157, 0.354, 0.332]
84.0


In [117]:
# Graphique satisfaction emploi

donnees = dict(zip(critere, pourcentage_satisfaction_formation))

sortie = pandas.DataFrame(donnees)

sortie.insert(0, "Rang", note)

sortie["Satisfaction sous-générale"] = pourcentage_satisfaction_formation_total

sortie["Satisfaction minimum"] = [satisfaction_min] * 4

sortie["Nombre répondants"] = [max(tab_total_critere)] * 4

sortie.head(10)

,Rang,Contenu de l'emploi,Perspectives de carrière,Satisfaction sous-générale,Satisfaction minimum,Nombre répondants
0,À améliorer,0.113,0.202,0.157,84.0,115
1,Satisfaisant,0.183,0.132,0.157,84.0,115
2,Bien,0.322,0.386,0.354,84.0,115
3,Très bien,0.383,0.281,0.332,84.0,115


In [118]:
# Modification fichier excel

sheet_name = "graph_18-1"

sauvegarde_excel([sortie], fichier_forme, sheet_name)

In [119]:
# Calcul satisfaction emploi 2

critere = ["Conditions de travail",
           "Relations",
           "Autonomie",
            "Rémunération"]

nom_colonne = ["93. EmploiSatisfactionConditionsTravail",
               "94. EmploiSatisfactionRelationsCollegues",
               "96. EmploiSatisfactionAutonomie", 
              "95. EmploiSatisfactionRemuneration"]

note = ["Très insatisfaisant", "Insatisfaisant", "Ni satisfaisant ni insatisfaisant", "Satisfaisant", "Très satisfaisant"]

tab_note = [[0 for _ in range(len(note))] for _ in range(len(critere))]

for i in range(len(critere)):
    for index, row in D.iterrows():
        if isinstance(row[nom_colonne[i]], float) and np.isnan(row[nom_colonne[i]]):
            continue
        tab_note[i][note.index(row[nom_colonne[i]])] += 1

#Total par critère (adéquation formation, ...)
tab_total_critere = [0] * len(critere)

for i in range(len(critere)):
    for j in range(len(note)):
        tab_total_critere[i] += tab_note[i][j]
        
#Total par note (très bien, ...)        
tab_total_note = [0] * len(note)

for i in range(len(note)):
    for j in range(len(critere)):
        tab_total_note[i] += tab_note[j][i]

print("effectif : ", tab_note)
print("total par notes : ", tab_total_note)
print("total par critères :", tab_total_critere)

# Pourcentage satisfaction emploi 2

pourcentage_satisfaction_formation = [[0 for _ in range(len(note))] for _ in range(len(critere))]

for i in range(len(critere)):
    for j in range(len(note)):
        pourcentage_satisfaction_formation[i][j] = round(tab_note[i][j] / tab_total_critere[i], 3)

print("pourcentage : ", pourcentage_satisfaction_formation)

# Pourcentage satisfaction emploi total 2

pourcentage_satisfaction_emploi = [0] * len(note)
tab_total_note = [0] * len(note)

for index, row in D.iterrows():
    if isinstance(row["92. EmploiSatisfaction"], float) and np.isnan(row["92. EmploiSatisfaction"]):
        continue
    tab_total_note[note.index(row["92. EmploiSatisfaction"])] += 1

total_note = sum(tab_total_note)

print(tab_total_note)
print(total_note)

for i in range(len(note)):
    pourcentage_satisfaction_emploi[i] = round(tab_total_note[i] / total_note, 3)

print(pourcentage_satisfaction_emploi)
    
satisfaction_min = round(sum(pourcentage_satisfaction_emploi[3:]) * 100, 0)

print(satisfaction_min)

effectif :  [[1, 4, 19, 32, 58], [1, 0, 10, 33, 70], [3, 11, 24, 39, 37], [9, 16, 30, 36, 24]]
total par notes :  [14, 31, 83, 140, 189]
total par critères : [114, 114, 114, 115]
pourcentage :  [[0.009, 0.035, 0.167, 0.281, 0.509], [0.009, 0.0, 0.088, 0.289, 0.614], [0.026, 0.096, 0.211, 0.342, 0.325], [0.078, 0.139, 0.261, 0.313, 0.209]]
[2, 8, 16, 41, 18]
85
[0.024, 0.094, 0.188, 0.482, 0.212]
69.0


In [120]:
# Graphique satisfaction emploi 2

donnees = dict(zip(critere, pourcentage_satisfaction_formation))

sortie = pandas.DataFrame(donnees)

sortie.insert(0, "Rang", note)

sortie["Satisfaction générale"] = pourcentage_satisfaction_emploi

sortie["Satisfaction min"] = satisfaction_min

sortie.head(10) 

,Rang,Conditions de travail,Relations,Autonomie,Rémunération,Satisfaction générale,Satisfaction min
0,Très insatisfaisant,0.009,0.009,0.026,0.078,0.024,69.0
1,Insatisfaisant,0.035,0.000,0.096,0.139,0.094,69.0
2,Ni satisfaisant ni insatisfaisant,0.167,0.088,0.211,0.261,0.188,69.0
3,Satisfaisant,0.281,0.289,0.342,0.313,0.482,69.0
4,Très satisfaisant,0.509,0.614,0.325,0.209,0.212,69.0


In [121]:
# Modification fichier excel


sheet_name = "graph_18-2"

sauvegarde_excel([sortie], fichier_forme, sheet_name)